In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import missingno

In [105]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_csv('data\premier league\england-premier-league-matches-2018-to-2019-stats.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 67 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   timestamp                            380 non-null    int64  
 1   date_GMT                             380 non-null    object 
 2   status                               380 non-null    object 
 3   attendance                           380 non-null    int64  
 4   home_team_name                       380 non-null    object 
 5   away_team_name                       380 non-null    object 
 6   referee                              380 non-null    object 
 7   Game Week                            380 non-null    int64  
 8   Pre-Match PPG (Home)                 380 non-null    float64
 9   Pre-Match PPG (Away)                 380 non-null    float64
 10  home_ppg                             380 non-null    float64
 11  away_ppg                        

In [5]:
data.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league
0,1691780400,Aug 11 2023 - 7:00pm,complete,21572.0,Burnley,Manchester City,Craig Pawson,1,0.0,0.0,0.53,2.32,0,3,3,2,0,2,NaN,"4,36,75",6,5,0,1,0,0,0,1,0,0,5,13,2,9,3,4,11,8,34,66,0.0,0.0,0.65,1.72,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,11.00,6.30,1.28,1.20,1.56,2.65,4.85,2.05,1.70,Turf Moor (Burnley),premier league
1,1691841600,Aug 12 2023 - 12:00pm,complete,59984.0,Arsenal,Nottingham Forest,Michael Oliver,1,0.0,0.0,2.47,0.84,2,1,3,2,2,0,"26,32",82,8,3,2,0,2,0,1,1,1,1,11,6,8,3,3,3,12,12,78,22,0.0,0.0,1.72,0.76,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.25,6.25,9.50,1.11,1.45,2.01,3.45,2.00,1.75,Emirates Stadium (London),premier league
2,1691848800,Aug 12 2023 - 2:00pm,complete,NaN,AFC Bournemouth,West Ham United,Peter Bankes,1,0.0,0.0,1.42,1.21,1,1,2,0,0,0,82,51,10,4,1,0,4,0,0,1,1,3,13,13,6,4,7,9,8,14,63,37,0.0,0.0,1.66,1.39,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.63,3.06,2.35,1.29,1.89,3.00,5.75,1.70,2.05,Vitality Stadium (Bournemouth- Dorset),premier league
3,1691848800,Aug 12 2023 - 2:00pm,complete,NaN,Brighton & Hove Albion,Luton Town,David Coote,1,0.0,0.0,1.58,0.53,4,1,5,1,1,0,"36,71,85,90'5",81,6,7,2,0,2,0,2,0,0,2,23,8,13,4,10,4,4,8,71,29,0.0,0.0,2.91,1.06,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.34,4.70,7.00,1.18,1.58,2.38,4.90,2.00,1.75,The American Express Community Stadium (Falmer...,premier league
4,1691848800,Aug 12 2023 - 2:00pm,complete,NaN,Everton,Fulham,Stuart Attwell,1,0.0,0.0,1.47,0.95,0,1,1,0,0,0,NaN,73,10,4,0,0,2,0,0,0,1,1,16,9,10,3,6,6,12,6,41,59,0.0,0.0,2.09,1.05,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.32,3.10,2.89,1.29,2.08,3.40,7.00,1.80,1.95,Goodison Park (Liverpool),premier league


In [11]:
data = clean_data(data)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_2548\1461863474.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date_GMT'] = pd.to_datetime(data['date_GMT'])


In [33]:
data_folder = './data'

for entry in os.listdir(data_folder):
    entry_path = os.path.join(data_folder, entry)
    if os.path.isdir(entry_path):
        for file_name in os.listdir(entry_path):
            if file_name.endswith('.csv'):
                file_path = os.path.join(entry_path, file_name)
                df = pd.read_csv(file_path)
                df['league'] = entry
                df.to_csv(file_path, index=False)

In [34]:
data = pd.DataFrame()

for folder in os.listdir('./data'):
    if folder.endswith('csv'):
        continue
    for file in os.listdir(f'./data/{folder}'):
        df = pd.read_csv(f'./data/{folder}/{file}')
        data = pd.concat([data, df], ignore_index=True)

In [35]:
data.isna().sum()

timestamp                                  0
date_GMT                                   0
status                                     0
attendance                             30917
home_team_name                             0
away_team_name                             0
referee                                31947
Game Week                               6853
Pre-Match PPG (Home)                       0
Pre-Match PPG (Away)                       0
home_ppg                                   0
away_ppg                                   0
home_team_goal_count                       0
away_team_goal_count                       0
total_goal_count                           0
total_goals_at_half_time                   0
home_team_goal_count_half_time             0
away_team_goal_count_half_time             0
home_team_goal_timings                 19417
away_team_goal_timings                 27086
home_team_corner_count                     0
away_team_corner_count                     0
home_team_

In [5]:
def clean_data(data):
    #transforming date variable
    data['date_GMT'] = pd.to_datetime(data['date_GMT'])
    data['date'] = pd.to_datetime(data['date_GMT'].dt.date)
    data['time'] = data['date_GMT'].dt.time
    
    #adding aggregated variables
    data['corners_total'] = data['home_team_corner_count'] + data['away_team_corner_count']
    data['yellow_cards_total'] = data['home_team_yellow_cards'] + data['away_team_yellow_cards']
    data['red_cards_total'] = data['home_team_red_cards'] + data['away_team_red_cards']
    data['cards_total'] = data['yellow_cards_total'] + data['red_cards_total']
    data['shots_total'] = data['home_team_shots'] + data['away_team_shots']
    data['shots_on_target_total'] = data['home_team_shots_on_target'] + data['away_team_shots_on_target']
    data['shots_off_target_total'] = data['home_team_shots_off_target'] + data['away_team_shots_off_target']
    data['fouls_total'] = data['home_team_fouls'] + data['away_team_fouls']
    
    #getting stadium names without city in brackets
    # data['base_name'] = data['stadium_name'].str.replace(r" \(.*\)$", "", regex=True)

    # city_map = data[data['stadium_name'].str.contains(r"\(.*\)")].copy()
    # city_map['city'] = city_map['stadium_name'].str.extract(r"\((.*?)\)")[0]
    # city_map = city_map.groupby('base_name')['city'].agg(pd.Series.mode).to_dict()

    # #filling stadium names without city in brackets
    # data['normalized_stadium'] = data.apply(lambda row: f"{row['base_name']} ({city_map.get(row['base_name'], 'Unknown')})" if '(' not in row['stadium_name'] else row['stadium_name'], axis=1)

    #dropping unnecessary features
    data.drop(['timestamp', 'status', 'home_team_goal_timings', 'away_team_goal_timings', 'date_GMT'], axis=1, inplace=True)
    
    #setting one of the target variable - result of the game
    data['result'] = np.where(data['home_team_goal_count'] == data['away_team_goal_count'], 0, np.where(data['home_team_goal_count'] > data['away_team_goal_count'], 1, 2))
    
    def assign_season(date):
        year = date.year
        if date.month >= 8:  # Sezon zaczyna się w sierpniu
            return f'{str(year)[2:]}/{str(year+1)[2:]}'
        else:
            return f'{str(year-1)[2:]}/{str(year)[2:]}'
    
    data['season'] = data['date'].apply(assign_season)
    
    return data

In [6]:
data = clean_data(data)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_29968\1461863474.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date_GMT'] = pd.to_datetime(data['date_GMT'])


In [40]:
data.shape

(83142, 74)

In [41]:
data.head()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season
0,8016.0,Austria Wien,Admira,NaN,1.0,0.0,0.0,1.61,0.67,2,0,2,0,0,0,6,7,2,0,5,0,0,2,1,4,13,13,7,6,6,7,9,17,53,47,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.36,5.15,9.65,0.0,0.0,0.0,0.0,0.0,0.0,Generali Arena (Wien),austria,2013-07-20,14:30:00,13,7,0,7,26,13,13,26,1,12/13
1,2840.0,Grödig,Ried,NaN,1.0,0.0,0.0,1.44,0.94,0,0,0,0,0,0,6,11,8,0,5,0,5,3,2,3,8,16,4,10,4,6,20,24,49,51,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.70,3.42,2.73,0.0,0.0,0.0,0.0,0.0,0.0,DAS.GOLDBERG Stadion,austria,2013-07-20,17:00:00,17,13,0,13,24,14,10,44,0,12/13
2,2696.0,Wiener Neustadt,Salzburg,NaN,1.0,0.0,0.0,1.11,1.94,1,5,6,3,1,2,3,10,4,0,1,0,0,4,0,1,5,18,2,6,3,12,13,16,52,48,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,6.97,4.51,1.51,0.0,0.0,0.0,0.0,0.0,0.0,Teddybären&Plüsch-Stadion,austria,2013-07-20,17:00:00,13,5,0,5,23,8,15,29,2,12/13
3,6413.0,Wolfsberger AC,Rapid Wien,NaN,1.0,0.0,0.0,1.33,1.33,2,2,4,2,0,2,6,3,2,0,4,0,0,2,2,2,7,13,3,6,4,7,8,26,51,49,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.29,3.37,2.33,0.0,0.0,0.0,0.0,0.0,0.0,Lavanttal Arena,austria,2013-07-20,17:00:00,9,6,0,6,20,9,11,34,0,12/13
4,7406.0,Wacker Innsbruck,Sturm Graz,NaN,1.0,0.0,0.0,0.94,1.44,2,2,4,0,0,0,7,1,2,0,4,0,0,2,0,4,15,5,5,3,10,2,21,20,48,52,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.54,3.37,2.93,0.0,0.0,0.0,0.0,0.0,0.0,Tivoli Stadion Tirol,austria,2013-07-21,14:30:00,8,6,0,6,20,8,12,41,0,12/13


In [42]:
data.isna().sum()

attendance                             30917
home_team_name                             0
away_team_name                             0
referee                                31947
Game Week                               6853
Pre-Match PPG (Home)                       0
Pre-Match PPG (Away)                       0
home_ppg                                   0
away_ppg                                   0
home_team_goal_count                       0
away_team_goal_count                       0
total_goal_count                           0
total_goals_at_half_time                   0
home_team_goal_count_half_time             0
away_team_goal_count_half_time             0
home_team_corner_count                     0
away_team_corner_count                     0
home_team_yellow_cards                     0
home_team_red_cards                        0
away_team_yellow_cards                     0
away_team_red_cards                        0
home_team_first_half_cards                 0
home_team_

# Attendance

In [13]:
# Pierwsze wypełnienie: średnia dla gospodarza i sezonu
data['attendance'] = data['attendance'].fillna(data.groupby(['home_team_name', 'season'])['attendance'].transform('mean'))

In [15]:
data['attendance'].isna().sum()

16647

In [16]:
# Drugie wypełnienie: średnia dla ligi i sezonu
data['attendance'] = data['attendance'].fillna(data.groupby(['league', 'season'])['attendance'].transform('mean'))

In [18]:
data['attendance'].isna().sum()

8968

In [19]:
# Trzecie wypełnienie: średnia dla ligi na przestzeni całego datasetu
data['attendance'] = data['attendance'].fillna(data.groupby(['league'])['attendance'].transform('mean'))

In [20]:
data['attendance'].isna().sum()

0

# Rolling variables

In [21]:
# Funkcja do obliczania średniej strzałów dla drużyny (zarówno dom jak i wyjazd)
def rolling_shots_avg(data, team_col, shots_col):
    return data.groupby(team_col)[shots_col].transform(lambda x: x.rolling(window=5, min_periods=1).mean())

In [22]:
# Obliczanie średniej liczby strzałów dla drużyn domowych i wyjazdowych
data['home_team_shots_rolling_avg'] = rolling_shots_avg(data, 'home_team_name', 'home_team_shots')
data['away_team_shots_rolling_avg'] = rolling_shots_avg(data, 'away_team_name', 'away_team_shots')

In [24]:
# Tworzenie nowych kolumn dla sumy kartek
data['home_team_total_cards'] = data['home_team_yellow_cards'] + data['home_team_red_cards']
data['away_team_total_cards'] = data['away_team_yellow_cards'] + data['away_team_red_cards']

In [25]:
# Funkcja do obliczania średniej ruchomej dla danych statystyk
def rolling_avg(data, team_col, stat_col):
    return data.groupby(team_col)[stat_col].transform(lambda x: x.rolling(window=5, min_periods=1).mean())

# Obliczanie średnich dla goli, rożnych i kartek
# Dla goli
data['home_team_goal_count_rolling_avg'] = rolling_avg(data, 'home_team_name', 'home_team_goal_count')
data['away_team_goal_count_rolling_avg'] = rolling_avg(data, 'away_team_name', 'away_team_goal_count')

# Dla rzutów rożnych
data['home_team_corner_count_rolling_avg'] = rolling_avg(data, 'home_team_name', 'home_team_corner_count')
data['away_team_corner_count_rolling_avg'] = rolling_avg(data, 'away_team_name', 'away_team_corner_count')

# Dla kartek
data['home_team_cards_rolling_avg'] = rolling_avg(data, 'home_team_name', 'home_team_total_cards')
data['away_team_cards_rolling_avg'] = rolling_avg(data, 'away_team_name', 'away_team_total_cards')

In [26]:
data.head()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,home_team_shots_rolling_avg,away_team_shots_rolling_avg,home_team_total_cards,away_team_total_cards,home_team_goal_count_rolling_avg,away_team_goal_count_rolling_avg,home_team_corner_count_rolling_avg,away_team_corner_count_rolling_avg,home_team_cards_rolling_avg,away_team_cards_rolling_avg
0,8016.0,Austria Wien,Admira,NaN,1.0,0.0,0.0,1.61,0.67,2,0,2,0,0,0,6,7,2,0,5,0,0,2,1,4,13,13,7,6,6,7,9,17,53,47,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.36,5.15,9.65,0.0,0.0,0.0,0.0,0.0,0.0,Generali Arena (Wien),austria,2013-07-20,14:30:00,13,7,0,7,26,13,13,26,1,12/13,13.0,13.0,2,5,2.0,0.0,6.0,7.0,2.0,5.0
1,2840.0,Grödig,Ried,NaN,1.0,0.0,0.0,1.44,0.94,0,0,0,0,0,0,6,11,8,0,5,0,5,3,2,3,8,16,4,10,4,6,20,24,49,51,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.70,3.42,2.73,0.0,0.0,0.0,0.0,0.0,0.0,DAS.GOLDBERG Stadion,austria,2013-07-20,17:00:00,17,13,0,13,24,14,10,44,0,12/13,8.0,16.0,8,5,0.0,0.0,6.0,11.0,8.0,5.0
2,2696.0,Wiener Neustadt,Salzburg,NaN,1.0,0.0,0.0,1.11,1.94,1,5,6,3,1,2,3,10,4,0,1,0,0,4,0,1,5,18,2,6,3,12,13,16,52,48,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,6.97,4.51,1.51,0.0,0.0,0.0,0.0,0.0,0.0,Teddybären&Plüsch-Stadion,austria,2013-07-20,17:00:00,13,5,0,5,23,8,15,29,2,12/13,5.0,18.0,4,1,1.0,5.0,3.0,10.0,4.0,1.0
3,6413.0,Wolfsberger AC,Rapid Wien,NaN,1.0,0.0,0.0,1.33,1.33,2,2,4,2,0,2,6,3,2,0,4,0,0,2,2,2,7,13,3,6,4,7,8,26,51,49,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.29,3.37,2.33,0.0,0.0,0.0,0.0,0.0,0.0,Lavanttal Arena,austria,2013-07-20,17:00:00,9,6,0,6,20,9,11,34,0,12/13,7.0,13.0,2,4,2.0,2.0,6.0,3.0,2.0,4.0
4,7406.0,Wacker Innsbruck,Sturm Graz,NaN,1.0,0.0,0.0,0.94,1.44,2,2,4,0,0,0,7,1,2,0,4,0,0,2,0,4,15,5,5,3,10,2,21,20,48,52,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.54,3.37,2.93,0.0,0.0,0.0,0.0,0.0,0.0,Tivoli Stadion Tirol,austria,2013-07-21,14:30:00,8,6,0,6,20,8,12,41,0,12/13,15.0,5.0,2,4,2.0,2.0,7.0,1.0,2.0,4.0


# Game week

In [42]:
df = pd.read_csv('./data/premier league/england-premier-league-matches-2007-to-2008-stats.csv')

In [43]:
df

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league
0,1186832700,Aug 11 2007 - 11:45am,complete,NaN,Sunderland,Tottenham Hotspur,NaN,NaN,0.00,0.00,1.58,0.89,1,0,1,0,0,0,90'3,NaN,-1,-1,1,0,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stadium of Light (Sunderland),premier league
1,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,West Ham United,Manchester City,NaN,NaN,0.00,0.00,1.47,0.95,0,2,2,1,0,1,NaN,"18,87",-1,-1,0,0,4,0,0,0,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Boleyn Ground (London),premier league
2,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,Middlesbrough,Blackburn Rovers,NaN,NaN,0.00,0.00,1.37,1.42,1,2,3,1,1,0,30,"63,79",-1,-1,3,0,4,0,2,1,0,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Riverside Stadium (Middlesbrough),premier league
3,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,Everton,Wigan Athletic,NaN,NaN,0.00,0.00,1.95,0.58,2,1,3,1,1,0,"26,75",80,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Goodison Park (Liverpool),premier league
4,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,Derby County,Portsmouth,NaN,NaN,0.00,0.00,0.42,1.47,2,2,4,2,1,1,"4,84","27,83",-1,-1,1,0,2,0,0,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pride Park Stadium (Derby),premier league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1210514400,May 11 2008 - 2:00pm,complete,NaN,West Ham United,Aston Villa,NaN,NaN,1.50,1.44,1.47,1.42,2,2,4,2,1,1,"8,88","14,58",-1,-1,2,0,2,0,2,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,2.94,61,86,56,28,9,28,78,48,86,0.0,3.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Boleyn Ground (London),premier league
376,1210514400,May 11 2008 - 2:00pm,complete,NaN,Tottenham Hotspur,Liverpool,NaN,NaN,1.61,1.72,1.53,1.79,0,2,2,0,0,0,NaN,"69,74",-1,-1,1,0,1,0,0,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,3.20,58,84,58,39,17,28,75,56,86,0.0,2.72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,White Hart Lane (London),premier league
377,1210514400,May 11 2008 - 2:00pm,complete,NaN,Sunderland,Arsenal,NaN,NaN,1.67,1.83,1.58,1.89,0,1,1,1,0,1,NaN,24,-1,-1,1,0,1,0,0,1,0,1,-1,-1,-1,-1,-1,-1,-1,

In [46]:
import pandas as pd

# Funkcja do przetwarzania jednego pliku
def process_season(df):
    df['date_GMT'] = pd.to_datetime(df['date_GMT'], format='%b %d %Y - %I:%M%p')
    df = df.sort_values(by='date_GMT')
    if 'Game Week' not in df.columns or df['Game Week'].isna().any():
        df['Game Week'] = df['date_GMT'].rank(method='dense').astype(int)
    return df

# Przykład użycia funkcji
processed_season_df = process_season(df)
processed_season_df.to_csv('path_to_processed_season_csv_file.csv', index=False)

In [47]:
process_season = pd.read_csv('path_to_processed_season_csv_file.csv')

In [48]:
process_season

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league
0,1186832700,2007-08-11 11:45:00,complete,NaN,Sunderland,Tottenham Hotspur,NaN,1,0.00,0.00,1.58,0.89,1,0,1,0,0,0,90'3,NaN,-1,-1,1,0,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stadium of Light (Sunderland),premier league
1,1186840800,2007-08-11 14:00:00,complete,NaN,West Ham United,Manchester City,NaN,2,0.00,0.00,1.47,0.95,0,2,2,1,0,1,NaN,"18,87",-1,-1,0,0,4,0,0,0,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Boleyn Ground (London),premier league
2,1186840800,2007-08-11 14:00:00,complete,NaN,Middlesbrough,Blackburn Rovers,NaN,2,0.00,0.00,1.37,1.42,1,2,3,1,1,0,30,"63,79",-1,-1,3,0,4,0,2,1,0,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Riverside Stadium (Middlesbrough),premier league
3,1186840800,2007-08-11 14:00:00,complete,NaN,Everton,Wigan Athletic,NaN,2,0.00,0.00,1.95,0.58,2,1,3,1,1,0,"26,75",80,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Goodison Park (Liverpool),premier league
4,1186840800,2007-08-11 14:00:00,complete,NaN,Derby County,Portsmouth,NaN,2,0.00,0.00,0.42,1.47,2,2,4,2,1,1,"4,84","27,83",-1,-1,1,0,2,0,0,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pride Park Stadium (Derby),premier league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1210514400,2008-05-11 14:00:00,complete,NaN,Wigan Athletic,Manchester United,NaN,200,1.61,1.78,1.53,1.84,0,2,2,1,0,1,NaN,"33,80",-1,-1,3,0,2,0,0,3,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,2.28,42,64,33,17,9,28,70,31,72,0.0,3.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DW Stadium (Wigan),premier league
376,1210514400,2008-05-11 14:00:00,complete,NaN,Portsmouth,Fulham,NaN,200,1.61,0.72,1.53,0.84,0,1,1,0,0,0,NaN,76,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,2.25,45,64,31,20,11,25,58,31,70,0.0,3.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fratton Park (Portsmouth),premier league
377,1210514400,2008-05-11 14:00:00,complete,NaN,Derby County,Reading,NaN,200,0.44,0.39,0.42,0.53,0,4,4,1,0,1,NaN,"15,61,69,90'1",-1,-1,0,0,1,0,0,0,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0

## Variables creation

In [46]:
data['time'] = pd.to_datetime(data['time'], format='%H:%M').dt.time
data['pora dnia'] = data['time'].apply(lambda x: 'rano' if x < pd.Timestamp("12:00").time() else ('dzień' if x < pd.Timestamp("18:00").time() else 'wieczór'))

ValueError: unconverted data remains when parsing with format "%H:%M": ":00", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [49]:
data['date'] = pd.to_datetime(data['date'])
data['runda'] = data['date'].dt.month.apply(lambda x: 'wiosna' if x >= 1 and x <= 6 else 'jesień')

# Pipelines

In [319]:
df = pd.read_csv('data\premier league\england-premier-league-matches-2018-to-2019-stats.csv')

In [320]:
df = clean_data(df)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_29968\1461863474.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date_GMT'] = pd.to_datetime(data['date_GMT'])


In [321]:
df.sort_values('date', inplace=True)

# Goals scored before every game

In [322]:
# to jest elegancko - teraz to musze wykorzystać do pozostałych agregatów

# Tworzenie DataFrame z bramkami zdobywanymi zarówno w domu, jak i na wyjeździe
goals = pd.concat([
    df[['date', 'home_team_name', 'home_team_goal_count']].rename(columns={'home_team_name': 'team', 'home_team_goal_count': 'goals'}),
    df[['date', 'away_team_name', 'away_team_goal_count']].rename(columns={'away_team_name': 'team', 'away_team_goal_count': 'goals'})
])

# Sortowanie danych według daty, aby kumulatywne sumy były poprawne
goals.sort_values('date', inplace=True)

# Obliczanie kumulatywnej liczby bramek dla każdej drużyny
goals['cumulative_goals'] = goals.groupby('team')['goals'].cumsum()

# Usunięcie bieżących bramek z kumulatywnej sumy, aby liczyć tylko bramki przed bieżącym meczem
goals['cumulative_goals'] -= goals['goals']

# Powrót do oryginalnego DataFrame z wynikami meczów i dodanie obliczonych kumulatywnych bramek
df = df.merge(goals[['date', 'team', 'cumulative_goals']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_goals': 'cumulative_goals_scored_home_team_before_game'}).drop('team', axis=1)
df = df.merge(goals[['date', 'team', 'cumulative_goals']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_goals': 'cumulative_goals_scored_away_team_before_game'}).drop('team', axis=1)

# Goals conceded before every game

In [323]:
import pandas as pd

# Tworzenie DataFrame z bramkami straconymi zarówno w domu, jak i na wyjeździe
goals_conceded = pd.concat([
    df[['date', 'home_team_name', 'away_team_goal_count']].rename(columns={'home_team_name': 'team', 'away_team_goal_count': 'goals_conceded'}),
    df[['date', 'away_team_name', 'home_team_goal_count']].rename(columns={'away_team_name': 'team', 'home_team_goal_count': 'goals_conceded'})
])

# Sortowanie danych według daty
goals_conceded.sort_values('date', inplace=True)

# Obliczanie kumulatywnej liczby bramek straconych dla każdej drużyny
goals_conceded['cumulative_goals_conceded'] = goals_conceded.groupby('team')['goals_conceded'].cumsum()

# Usunięcie bieżących bramek straconych z kumulatywnej sumy, aby liczyć tylko bramki przed bieżącym meczem
goals_conceded['cumulative_goals_conceded'] -= goals_conceded['goals_conceded']

In [324]:
# Dodanie kumulatywnych bramek straconych do oryginalnego DataFrame
df = df.merge(goals_conceded[['date', 'team', 'cumulative_goals_conceded']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_goals_conceded': 'cumulative_goals_conceded_home_team_before_game'}).drop('team', axis=1)
df = df.merge(goals_conceded[['date', 'team', 'cumulative_goals_conceded']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_goals_conceded': 'cumulative_goals_conceded_away_team_before_game'}).drop('team', axis=1)

In [280]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_goals_scored_home_team_before_game,cumulative_goals_scored_away_team_before_game,cumulative_goals_conceded_home_team_before_game,cumulative_goals_conceded_away_team_before_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,51,63,48,39
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,87,47,22,44
377,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,1.89,0.74,0,2,2,1,0,1,11,2,3,0,3,0,1,2,1,2,21,11,11,5,10,6,9,6,73,27,1.67,1.10,2.67,1.31,2.72,56,78,56,31,20,25,64,53,78,9.77,3.50,1.28,6.2,10.0,1.12,1.41,2.00,3.60,1.74,2.05,Old Trafford (Manchester),premier league,2019-05-12,14:00:00,13,6,0,6,32,16,16,15,2,18/19,65,32,52,69
378,30367,Southampton,Huddersfield Town,Lee Probert,38,1.22,0.33,1.21,0.37,1,1,2,1,1,0,4,3,0,0,1,0,0,0,0,1,7,8,4,4,3,4,8,6,53,47,1.46,1.11,1.15,1.04,3.06,59,84,61,39,17,45,67,53,78,9.50,4.00,1.41,4.9,7.5,1.19,1.61,2.50,4.75,1.80,2.00,St. Mary's Stadium (Southampton- Hampshire),premier league,2019-05-12,14:00:00,7,1,0,1,15,8,7,14,0,18/19,44,21,64,75
379,20067,Watford,West Ham United,Chris Kavanagh,38,1.50,1.00,1.42,1.11,1,4,5,2,0,2,7,2,1,1,0,0,0,2,0,0,11,15,9,10,2,5,10,10,48,52,1.30,1.18,1.81,2.01,2.56,50,83,53,17,6,28,72,42,86,9.00,4.11,2.20,3.7,3.1,1.16,1.54,2.35,4.00,1.48,2.55,Vicarage Road (Watford),premier league,2019-05-12,14:00:00,9,1,1,2,26,19,7,20,2,18/19,51,48,55,54


# Average goals scored and conceded per game before game

In [326]:
# Dodanie kolumn z goli zdobytymi przez gospodarzy i gości
df['home_goals_scored'] = df['home_team_goal_count']
df['away_goals_scored'] = df['away_team_goal_count']

# Dodanie kolumn z goli straconymi przez gospodarzy i gości
df['home_goals_conceded'] = df['away_team_goal_count']
df['away_goals_conceded'] = df['home_team_goal_count']

In [327]:
# Tworzenie DataFrame z danymi dotyczącymi zdobytych i straconych goli
goals_data = pd.concat([
    df[['date', 'home_team_name', 'home_goals_scored', 'home_goals_conceded']].rename(columns={'home_team_name': 'team', 'home_goals_scored': 'goals_scored', 'home_goals_conceded': 'goals_conceded'}),
    df[['date', 'away_team_name', 'away_goals_scored', 'away_goals_conceded']].rename(columns={'away_team_name': 'team', 'away_goals_scored': 'goals_scored', 'away_goals_conceded': 'goals_conceded'})
])

# Sortowanie danych według daty
goals_data.sort_values('date', inplace=True)

In [328]:
# Obliczanie kumulatywnych sum goli zdobytych i straconych
goals_data['cumulative_goals_scored'] = goals_data.groupby('team')['goals_scored'].cumsum()
goals_data['cumulative_goals_conceded'] = goals_data.groupby('team')['goals_conceded'].cumsum()

# Liczenie liczby meczów
goals_data['games_played'] = goals_data.groupby('team').cumcount() + 1

# Obliczanie średniej liczby goli zdobytych i straconych na mecz
goals_data['average_goals_scored_per_game'] = goals_data['cumulative_goals_scored'] / goals_data['games_played']
goals_data['average_goals_conceded_per_game'] = goals_data['cumulative_goals_conceded'] / goals_data['games_played']

# Usuwamy ostatni mecz, aby uzyskać dane przed bieżącym meczem
goals_data['average_goals_scored_per_game_pre_game'] = goals_data.groupby('team')['average_goals_scored_per_game'].shift().fillna(0)
goals_data['average_goals_conceded_per_game_pre_game'] = goals_data.groupby('team')['average_goals_conceded_per_game'].shift().fillna(0)

In [329]:
df = df.merge(goals_data[['date', 'team', 'average_goals_scored_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_goals_scored_per_game_pre_game': 'home_team_average_goals_scored_per_game'}).drop('team', axis=1)
df = df.merge(goals_data[['date', 'team', 'average_goals_scored_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_goals_scored_per_game_pre_game': 'away_team_average_goals_scored_per_game'}).drop('team', axis=1)
df = df.merge(goals_data[['date', 'team', 'average_goals_conceded_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_goals_conceded_per_game_pre_game': 'home_team_average_goals_conceded_per_game'}).drop('team', axis=1)
df = df.merge(goals_data[['date', 'team', 'average_goals_conceded_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_goals_conceded_per_game_pre_game': 'away_team_average_goals_conceded_per_game'}).drop('team', axis=1)

In [330]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_goals_scored_home_team_before_game,cumulative_goals_scored_away_team_before_game,cumulative_goals_conceded_home_team_before_game,cumulative_goals_conceded_away_team_before_game,home_goals_scored,away_goals_scored,home_goals_conceded,away_goals_conceded,home_team_average_goals_scored_per_game,away_team_average_goals_scored_per_game,home_team_average_goals_conceded_per_game,away_team_average_goals_conceded_per_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,51,63,48,39,0,0,0,0,1.378378,1.702703,1.297297,1.054054
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,87,47,22,44,2,0,0,2,2.351351,1.270270,0.594595,1.189189
377,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,1.89,0.74,0,2,2,1,0,1,11,2,3,0,3,0,1,2,1,2,21,11,11,5,10,6,9,6,73,27,1.67,1.10,2.67,1.31,2.72,56,78,56,31,20,25,64,53,78,9.77,3.50,1.28,6.2,10.0,1.12,1.41,2.00,3.60,1.74,2.05,Old Trafford (Manchester),premier league,2019-05-12,14:00:00,13,6,0,6,32,16,16,15,2,18/19,65,32,52,69,0,2,2,0,1.756757,0.864865,1.405405,1.864865
378,30367,Southampton,Huddersfield Town,Lee Probert,38,1.22,0.33,1.21,0.37,1,1,2,1,1,0,4,3,0,0,1,0,0,0,0,1,7,8,4,4,3,4,8,6,53,47,1.46,1.11,1.15,1.04,3.06,59,84,61,39,17,45,67,53,78,9.50,4.00,1.41,4.9,7.5,1.19,1.61,2.50,4.75,1.80,2.00,St. Mary's Stadium (Southampton- Hampshire),premier league,2019-05-12,14:00:00,7,1,0,1,15,8,7,14,0,18/19,44,21,64,75,1,1,1,1,1.189189,0.567568,1.729730,2.027027
379,20067,Watford,West Ham United,Chris Kavanagh,38,1.50,1.00,1.42,1.11,1,4,5,2,0,2,7,2,1,1,0,0,0,2,0,0,11,15,9,10,2,5,10,10,48,52,1.30,1.18,1.81,2.01,2.56,50,83,53,17,6,28,72,42,86,9.00,4.11,2.20,3.7,3.1,1.16,1.54,2.35,4.00,1.48,2.55,Vicarage Road (Watford),premier league,2019-05-12,14:00:00,9,1,1,2,26,19,7,20,2,18/19,51,48,55,54,1,4,4,1,1.378378,1.297297,1.486486,1.459459


# Average goals scored first half/second half

In [331]:
df['home_team_second_half_goals'] = df['home_team_goal_count'] - df['home_team_goal_count_half_time']
df['away_team_second_half_goals'] = df['away_team_goal_count'] - df['away_team_goal_count_half_time']

In [332]:
import pandas as pd

# Tworzenie DataFrame z danymi o golach zdobytych w obu połowach
goals_data_half = pd.concat([
    df[['date', 'home_team_name', 'home_team_goal_count_half_time', 'home_team_second_half_goals']].rename(columns={'home_team_name': 'team', 'home_team_goal_count_half_time': 'first_half_goals', 'home_team_second_half_goals': 'second_half_goals'}),
    df[['date', 'away_team_name', 'away_team_goal_count_half_time', 'away_team_second_half_goals']].rename(columns={'away_team_name': 'team', 'away_team_goal_count_half_time': 'first_half_goals', 'away_team_second_half_goals': 'second_half_goals'})
])

# Sortowanie danych według daty
goals_data_half.sort_values('date', inplace=True)


In [333]:
# Obliczanie kumulatywnych sum goli zdobytych w pierwszej i drugiej połowie
goals_data_half['cumulative_first_half_goals'] = goals_data_half.groupby('team')['first_half_goals'].cumsum()
goals_data_half['cumulative_second_half_goals'] = goals_data_half.groupby('team')['second_half_goals'].cumsum()

# Liczenie liczby meczów
goals_data_half['games_played'] = goals_data_half.groupby('team').cumcount() + 1

# Obliczanie średniej liczby goli na mecz w pierwszej i drugiej połowie
goals_data_half['average_first_half_goals_per_game'] = goals_data_half['cumulative_first_half_goals'] / goals_data_half['games_played']
goals_data_half['average_second_half_goals_per_game'] = goals_data_half['cumulative_second_half_goals'] / goals_data_half['games_played']

# Usuwamy ostatni mecz, aby uzyskać dane przed bieżącym meczem
goals_data_half['average_first_half_goals_per_game_pre_game'] = goals_data_half.groupby('team')['average_first_half_goals_per_game'].shift().fillna(0)
goals_data_half['average_second_half_goals_per_game_pre_game'] = goals_data_half.groupby('team')['average_second_half_goals_per_game'].shift().fillna(0)

In [334]:
df = df.merge(goals_data_half[['date', 'team', 'average_first_half_goals_per_game_pre_game', 'average_second_half_goals_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_first_half_goals_per_game_pre_game': 'home_team_average_first_half_goals_scored', 'average_second_half_goals_per_game_pre_game': 'home_team_average_second_half_goals_scored'}).drop('team', axis=1)
df = df.merge(goals_data_half[['date', 'team', 'average_first_half_goals_per_game_pre_game', 'average_second_half_goals_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_first_half_goals_per_game_pre_game': 'away_team_average_first_half_goals_scored', 'average_second_half_goals_per_game_pre_game': 'away_team_average_second_half_goals_scored'}).drop('team', axis=1)
# df.drop([''])

# Average goals conceded first half / second half per team

In [337]:
# Kolumny home_goals_conceded i away_goals_conceded są już obliczone wcześniej:
# df['home_goals_conceded'] = df['away_team_goal_count']
# df['away_goals_conceded'] = df['home_team_goal_count']
df['home_goals_conceded_first_half'] = df['away_team_goal_count_half_time']
df['away_goals_conceded_first_half'] = df['home_team_goal_count_half_time']
df['home_goals_conceded_second_half'] = df['away_goals_conceded'] - df['away_goals_conceded_first_half']
df['away_goals_conceded_second_half'] = df['home_goals_conceded'] - df['home_goals_conceded_first_half']

In [338]:
# Tworzenie DataFrame z danymi o bramkach straconych
conceded_goals_data = pd.concat([
    df[['date', 'home_team_name', 'home_goals_conceded_first_half', 'home_goals_conceded_second_half']].rename(columns={'home_team_name': 'team', 'home_goals_conceded_first_half': 'first_half_conceded', 'home_goals_conceded_second_half': 'second_half_conceded'}),
    df[['date', 'away_team_name', 'away_goals_conceded_first_half', 'away_goals_conceded_second_half']].rename(columns={'away_team_name': 'team', 'away_goals_conceded_first_half': 'first_half_conceded', 'away_goals_conceded_second_half': 'second_half_conceded'})
])

# Sortowanie danych według daty
conceded_goals_data.sort_values('date', inplace=True)

In [339]:
# Obliczanie kumulatywnych sum bramek straconych
conceded_goals_data['cumulative_first_half_conceded'] = conceded_goals_data.groupby('team')['first_half_conceded'].cumsum()
conceded_goals_data['cumulative_second_half_conceded'] = conceded_goals_data.groupby('team')['second_half_conceded'].cumsum()

# Liczenie liczby meczów
conceded_goals_data['games_played'] = conceded_goals_data.groupby('team').cumcount() + 1

# Obliczanie średniej liczby bramek straconych na mecz w pierwszej i drugiej połowie
conceded_goals_data['average_first_half_conceded_per_game'] = conceded_goals_data['cumulative_first_half_conceded'] / conceded_goals_data['games_played']
conceded_goals_data['average_second_half_conceded_per_game'] = conceded_goals_data['cumulative_second_half_conceded'] / conceded_goals_data['games_played']

# Usuwamy ostatni mecz, aby uzyskać dane przed bieżącym meczem
conceded_goals_data['average_first_half_conceded_per_game_pre_game'] = conceded_goals_data.groupby('team')['average_first_half_conceded_per_game'].shift().fillna(0)
conceded_goals_data['average_second_half_conceded_per_game_pre_game'] = conceded_goals_data.groupby('team')['average_second_half_conceded_per_game'].shift().fillna(0)

In [340]:
df = df.merge(conceded_goals_data[['date', 'team', 'average_first_half_conceded_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_first_half_conceded_per_game_pre_game': 'home_team_average_first_half_goals_conceded'}).drop('team', axis=1)
df = df.merge(conceded_goals_data[['date', 'team', 'average_second_half_conceded_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_second_half_conceded_per_game_pre_game': 'home_team_average_second_half_goals_conceded'}).drop('team', axis=1)
df = df.merge(conceded_goals_data[['date', 'team', 'average_first_half_conceded_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_first_half_conceded_per_game_pre_game': 'away_team_average_first_half_goals_conceded'}). drop('team', axis=1)
df = df.merge(conceded_goals_data[['date', 'team', 'average_second_half_conceded_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_second_half_conceded_per_game_pre_game': 'away_team_average_second_half_goals_conceded'}).drop('team', axis=1)

In [341]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_goals_scored_home_team_before_game,cumulative_goals_scored_away_team_before_game,cumulative_goals_conceded_home_team_before_game,cumulative_goals_conceded_away_team_before_game,home_goals_scored,away_goals_scored,home_goals_conceded,away_goals_conceded,home_team_average_goals_scored_per_game,away_team_average_goals_scored_per_game,home_team_average_goals_conceded_per_game,away_team_average_goals_conceded_per_game,home_team_second_half_goals,away_team_second_half_goals,home_team_average_first_half_goals,home_team_average_second_half_goals,away_team_average_first_half_goals,away_team_average_second_half_goals,home_goals_conceded_first_half,away_goals_conceded_first_half,home_goals_conceded_second_half,away_goals_conceded_second_half,home_team_average_first_half_goals_conceded,home_team_average_second_half_goals_conceded,away_team_average_first_half_goals_conceded,away_team_average_second_half_goals_conceded
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,51,63,48,39,0,0,0,0,1.378378,1.702703,1.297297,1.054054,0,0,0.405405,0.972973,0.729730,0.972973,0,0,0,0,0.729730,0.972973,0.459459,0.972973
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,87,47,22,44,2,0,0,2,2.351351,1.270270,0.594595,1.189189,1,0,1.000000,1.351351,0.459459,0.810811,0,1,1,0,0.270270,1.351351,0.567568,0.810811
377,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,1.89,0.74,0,2,2,1,0,1,11,2,3,0,3,0,1,2,1,2,21,11,11,5,10,6,9,6,73,27,1.67,1.10,2.67,1.31,2.72,56,78,56,31,20,25,64,53,78,9.77,3.50,1.28,6.2,10.0,1.12,1.41,2.00,3.60,1.74,2.05,Old Trafford (Manchester),premier league,2019-05-12,14:00:00,13,6,0,6,32,16,16,15,2,18/19,65,32,52,69,0,2,2,0,1.756757,0.864865,1.405405,1.864865,0,1,0.945946,0.810811,0.270270,0.594595,1,0,0,1,0.594595,0.810811,0.810811,0.594595
378,30367,Southampton,Huddersfield Town,Lee Probert,38,1.22,0.33,1.21,0.37,1,1,2,1,1,0,4,3,0,0,1,0,0,0,0,1,7,8,4,4,3,4,8,6,53,47,1.46,1.11,1.15,1.04,3.06,59,84,61,

# Average goals total per half before game

In [342]:
df['home_first_half_goals'] = df['home_team_goal_count_half_time']
df['home_second_half_goals'] = df['home_team_goal_count'] - df['home_team_goal_count_half_time']
df['away_first_half_goals'] = df['away_team_goal_count_half_time']
df['away_second_half_goals'] = df['away_team_goal_count'] - df['away_team_goal_count_half_time']

In [343]:
goals_data_half = pd.concat([
    df[['date', 'home_team_name', 'home_first_half_goals', 'home_second_half_goals']].rename(columns={'home_team_name': 'team', 'home_first_half_goals': 'first_half_goals', 'home_second_half_goals': 'second_half_goals'}),
    df[['date', 'away_team_name', 'away_first_half_goals', 'away_second_half_goals']].rename(columns={'away_team_name': 'team', 'away_first_half_goals': 'first_half_goals', 'away_second_half_goals': 'second_half_goals'})
])

# Sortowanie danych według daty
goals_data_half.sort_values('date', inplace=True)

In [344]:
goals_data_half['cumulative_first_half_goals'] = goals_data_half.groupby('team')['first_half_goals'].cumsum()
goals_data_half['cumulative_second_half_goals'] = goals_data_half.groupby('team')['second_half_goals'].cumsum()

# Liczenie liczby meczów
goals_data_half['games_played'] = goals_data_half.groupby('team').cumcount() + 1

# Obliczanie średniej liczby bramek zdobytych w pierwszej i drugiej połowie na mecz
goals_data_half['average_first_half_goals_per_game'] = goals_data_half['cumulative_first_half_goals'] / goals_data_half['games_played']
goals_data_half['average_second_half_goals_per_game'] = goals_data_half['cumulative_second_half_goals'] / goals_data_half['games_played']

In [345]:
goals_data_half['average_first_half_goals_per_game_pre_game'] = goals_data_half.groupby('team')['average_first_half_goals_per_game'].shift().fillna(0)
goals_data_half['average_second_half_goals_per_game_pre_game'] = goals_data_half.groupby('team')['average_second_half_goals_per_game'].shift().fillna(0)

In [346]:
df = df.merge(goals_data_half[['date', 'team', 'average_first_half_goals_per_game_pre_game', 'average_second_half_goals_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_first_half_goals_per_game_pre_game': 'home_team_average_first_half_goals_total', 'average_second_half_goals_per_game_pre_game': 'home_team_average_second_half_goals_total'}).drop('team', axis=1)
df = df.merge(goals_data_half[['date', 'team', 'average_first_half_goals_per_game_pre_game', 'average_second_half_goals_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_first_half_goals_per_game_pre_game': 'away_team_average_first_half_goals_total', 'average_second_half_goals_per_game_pre_game': 'away_team_average_second_half_goals_total'}).drop('team', axis=1)


In [347]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_goals_scored_home_team_before_game,cumulative_goals_scored_away_team_before_game,cumulative_goals_conceded_home_team_before_game,cumulative_goals_conceded_away_team_before_game,home_goals_scored,away_goals_scored,home_goals_conceded,away_goals_conceded,home_team_average_goals_scored_per_game,away_team_average_goals_scored_per_game,home_team_average_goals_conceded_per_game,away_team_average_goals_conceded_per_game,home_team_second_half_goals,away_team_second_half_goals,home_team_average_first_half_goals,home_team_average_second_half_goals,away_team_average_first_half_goals,away_team_average_second_half_goals,home_goals_conceded_first_half,away_goals_conceded_first_half,home_goals_conceded_second_half,away_goals_conceded_second_half,home_team_average_first_half_goals_conceded,home_team_average_second_half_goals_conceded,away_team_average_first_half_goals_conceded,away_team_average_second_half_goals_conceded,home_first_half_goals,home_second_half_goals,away_first_half_goals,away_second_half_goals,home_team_average_first_half_goals_total,home_team_average_second_half_goals_total,away_team_average_first_half_goals_total,away_team_average_second_half_goals_total
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,51,63,48,39,0,0,0,0,1.378378,1.702703,1.297297,1.054054,0,0,0.405405,0.972973,0.729730,0.972973,0,0,0,0,0.729730,0.972973,0.459459,0.972973,0,0,0,0,0.405405,0.972973,0.729730,0.972973
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,87,47,22,44,2,0,0,2,2.351351,1.270270,0.594595,1.189189,1,0,1.000000,1.351351,0.459459,0.810811,0,1,1,0,0.270270,1.351351,0.567568,0.810811,1,1,0,0,1.000000,1.351351,0.459459,0.810811
377,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,1.89,0.74,0,2,2,1,0,1,11,2,3,0,3,0,1,2,1,2,21,11,11,5,10,6,9,6,73,27,1.67,1.10,2.67,1.31,2.72,56,78,56,31,20,25,64,53,78,9.77,3.50,1.28,6.2,10.0,1.12,1.41,2.00,3.60,1.74,2.05,Old Trafford (Manchester),premier league,2019-05-

# Average total corners before game

In [13]:
import pandas as pd

# Tworzenie DataFrame z łączną liczbą rzutów rożnych zarówno w domu, jak i na wyjeździe
corners = pd.concat([
    df[['date', 'home_team_name', 'corners_total']].rename(columns={'home_team_name': 'team', 'corners_total': 'corners'}),
    df[['date', 'away_team_name', 'corners_total']].rename(columns={'away_team_name': 'team', 'corners_total': 'corners'})
])

# Sortowanie danych według daty, aby kumulatywne średnie były poprawne
corners.sort_values('date', inplace=True)

In [14]:
# Obliczanie sumy i liczby meczów
corners['cumulative_corners'] = corners.groupby('team')['corners'].cumsum()
corners['games_played'] = corners.groupby('team').cumcount() + 1  # Dodajemy 1, bo cumcount zaczyna od 0

# Obliczanie średniej kumulatywnej
corners['average_corners'] = corners['cumulative_corners'] / corners['games_played']

# Usuwamy ostatni mecz, aby uzyskać średnią przed bieżącym meczem
corners['average_corners_pre_game'] = corners.groupby('team')['average_corners'].shift().fillna(0)

In [15]:
# Dodanie średniej liczby rzutów rożnych do oryginalnego DataFrame
df = df.merge(corners[['date', 'team', 'average_corners_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_corners_pre_game': 'average_home_corners'}).drop('team', axis=1)
df = df.merge(corners[['date', 'team', 'average_corners_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_corners_pre_game': 'average_away_corners'}).drop('team', axis=1)

In [18]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,22,44,10.027027,10.270270
377,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,1.89,0.74,0,2,2,1,0,1,11,2,3,0,3,0,1,2,1,2,21,11,11,5,10,6,9,6,73,27,1.67,1.10,2.67,1.31,2.72,56,78,56,31,20,25,64,53,78,9.77,3.50,1.28,6.2,10.0,1.12,1.41,2.00,3.60,1.74,2.05,Old Trafford (Manchester),premier league,2019-05-12,14:00:00,13,6,0,6,32,16,16,15,2,18/19,52,69,10.081081,11.486486
378,30367,Southampton,Huddersfield Town,Lee Probert,38,1.22,0.33,1.21,0.37,1,1,2,1,1,0,4,3,0,0,1,0,0,0,0,1,7,8,4,4,3,4,8,6,53,47,1.46,1.11,1.15,1.04,3.06,59,84,61,39,17,45,67,53,78,9.50,4.00,1.41,4.9,7.5,1.19,1.61,2.50,4.75,1.80,2.00,St. Mary's Stadium (Southampton- Hampshire),premier league,2019-05-12,14:00:00,7,1,0,1,15,8,7,14,0,18/19,64,75,10.837838,8.783784
379,20067,Watford,West Ham United,Chris Kavanagh,38,1.50,1.00,1.42,1.11,1,4,5,2,0,2,7,2,1,1,0,0,0,2,0,0,11,15,9,10,2,5,10,10,48,52,1.30,1.18,1.81,2.01,2.56,50,83,53,17,6,28,72,42,86,9.00,4.11,2.20,3.7,3.1,1.16,1.54,2.35,4.00,1.48,2.55,Vicarage Road (Watford),premier league,2019-05-12,14:00:00,9,1,1,2,26,19,7,20,2,18/19,55,54,10.216216,10.432432


# Average corners by team before every game

In [19]:
import pandas as pd

# Tworzenie DataFrame z liczbą rzutów rożnych zarówno w domu, jak i na wyjeździe
corners = pd.concat([
    df[['date', 'home_team_name', 'home_team_corner_count']].rename(columns={'home_team_name': 'team', 'home_team_corner_count': 'corners'}),
    df[['date', 'away_team_name', 'away_team_corner_count']].rename(columns={'away_team_name': 'team', 'away_team_corner_count': 'corners'})
])

# Sortowanie danych według daty, aby kumulatywne średnie były poprawne
corners.sort_values('date', inplace=True)   

In [20]:
# Obliczanie sumy i liczby meczów
corners['cumulative_corners'] = corners.groupby('team')['corners'].cumsum()
corners['games_played'] = corners.groupby('team').cumcount() + 1  # Dodajemy 1, bo cumcount zaczyna od 0

# Obliczanie średniej kumulatywnej
corners['average_corners'] = corners['cumulative_corners'] / corners['games_played']

# Usuwamy ostatni mecz, aby uzyskać średnią przed bieżącym meczem
corners['average_corners_pre_game'] = corners.groupby('team')['average_corners'].shift().fillna(0)

In [21]:
# Dodanie średniej liczby rzutów rożnych do oryginalnego DataFrame
df = df.merge(corners[['date', 'team', 'average_corners_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_corners_pre_game': 'average_home_corners'}).drop('team', axis=1)
df = df.merge(corners[['date', 'team', 'average_corners_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_corners_pre_game': 'average_away_corners'}).drop('team', axis=1)

# Average yellow cards total before every game

In [22]:
import pandas as pd

# Tworzenie DataFrame z łączną liczbą żółtych kartek zarówno w domu, jak i na wyjeździe
cards = pd.concat([
    df[['date', 'home_team_name', 'yellow_cards_total']].rename(columns={'home_team_name': 'team', 'yellow_cards_total': 'yellow_cards'}),
    df[['date', 'away_team_name', 'yellow_cards_total']].rename(columns={'away_team_name': 'team', 'yellow_cards_total': 'yellow_cards'})
])

# Sortowanie danych według daty, aby kumulatywne średnie były poprawne
cards.sort_values('date', inplace=True)


In [23]:
# Obliczanie sumy i liczby meczów
cards['cumulative_yellow_cards'] = cards.groupby('team')['yellow_cards'].cumsum()
cards['games_played'] = cards.groupby('team').cumcount() + 1  # Dodajemy 1, bo cumcount zaczyna od 0

# Obliczanie średniej kumulatywnej
cards['average_yellow_cards'] = cards['cumulative_yellow_cards'] / cards['games_played']

# Usuwamy ostatni mecz, aby uzyskać średnią przed bieżącym meczem
cards['average_yellow_cards_pre_game'] = cards.groupby('team')['average_yellow_cards'].shift().fillna(0)


In [24]:
# Dodanie średniej liczby żółtych kartek do oryginalnego DataFrame
df = df.merge(cards[['date', 'team', 'average_yellow_cards_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_yellow_cards_pre_game': 'average_total_yellow_cards_in_home_team_games'}).drop('team', axis=1)
df = df.merge(cards[['date', 'team', 'average_yellow_cards_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_yellow_cards_pre_game': 'average_total_yellow_cards_in_away_team_games'}).drop('team', axis=1)

In [81]:
df['home_team_yellow_cards']

0      1
1      2
2      3
3      2
4      0
      ..
375    4
376    4
377    0
378    0
379    0
Name: home_team_yellow_cards, Length: 380, dtype: int64

# Average yellow cards by team before every game

In [25]:
import pandas as pd

# Tworzenie DataFrame z liczbą żółtych kartek dla gospodarzy i gości
cards = pd.concat([
    df[['date', 'home_team_name', 'home_team_yellow_cards']].rename(columns={'home_team_name': 'team', 'home_team_yellow_cards': 'yellow_cards'}),
    df[['date', 'away_team_name', 'away_team_yellow_cards']].rename(columns={'away_team_name': 'team', 'away_team_yellow_cards': 'yellow_cards'})
])

# Sortowanie danych według daty, aby kumulatywne średnie były poprawne
cards.sort_values('date', inplace=True)

In [26]:
# Obliczanie sumy i liczby meczów
cards['cumulative_yellow_cards'] = cards.groupby('team')['yellow_cards'].cumsum()
cards['games_played'] = cards.groupby('team').cumcount() + 1  # Dodajemy 1, bo cumcount zaczyna od 0

# Obliczanie średniej kumulatywnej
cards['average_yellow_cards'] = cards['cumulative_yellow_cards'] / cards['games_played']

# Usuwamy ostatni mecz, aby uzyskać średnią przed bieżącym meczem
cards['average_yellow_cards_pre_game'] = cards.groupby('team')['average_yellow_cards'].shift().fillna(0)

In [27]:
# Dodanie średniej liczby żółtych kartek do oryginalnego DataFrame
df = df.merge(cards[['date', 'team', 'average_yellow_cards_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_yellow_cards_pre_game': 'average_yellow_cards_by_home_team_in home_team_games'}).drop('team', axis=1)
df = df.merge(cards[['date', 'team', 'average_yellow_cards_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_yellow_cards_pre_game': 'average_yellow_cards_by_away_team_in_away_team_games'}).drop('team', axis=1)


In [28]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,22,44,10.027027,10.270270,6.621622,5.243243,2.297297,3.945946,1.054054,1.891892
377,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,1.89,0.74,0,2,2,1,0,1,11,2,3,0,3,0,1,2,1,2,21,11,11,5,10,6,9,6,73,27,1.67,1.10,2.67,1.31,2.72,56,78,56,31,20,25,64,53,78,9.77,3.50,1.28,6.2,10.0,1.12,1.41,2.00,3.60,1.74,2.05,Old Trafford (Manchester),premier league,2019-05-12,14:00:00,13,6,0,6,32,16,16,15,2,18/19,52,69,10.081081,11.486486,5.108108,4.486486,3.675676,2.945946,1.945946,1.702703
378,30367,Southampton,Huddersfield Town,Lee Probert,38,1.22,0.33,1.21,0.37,1,1,2,1,1,0,4,3,0,0,1,0,0,0,0,1,7,8,4,4,3,4,8,6,53,47,1.46,1.11,1.15,1.04,3.06,59,84,61,39,17,45,67,53,78,9.50,4.00,1.41,4.9,7.5,1.19,1.61,2.50,4.75,1.80,2.00,St. Mary's Stadium (Southampton- Hampshire),premier league,2019-05-12,14:00:00,7,1,0,1,15,8,7,14,0,18/19,64,75,10.837838,8.783784,5.135135,4.297297,3.621622,3.270270,1.972973,1.486486
379,20067,Watford,West Ham United,Chris Kavanagh,38,1.50,1.00,1.42,1.11,1,4,5,2,0,2,7,2,1,1,0,0,0,2,0,0,11,15,9,10,2,5,10,10,48,52,1.30,1.18,1.81,2.01,2.56,50,83,53,17,6,28,72,42,86,9.00,4.11,2.20,3.7,3.1,1.16,1.54,2.35,4.00,1.48,2.55,Vicarage Road (Watford),premier league,2019-05-12,14:00:00,9,1,1,2,26,19,7,20,2,18/19,55,54,10.216216,10.432432,4.594595,5.027027,3.432432,3.135135,2.081081,1.594595


# Red cards per team before game

In [93]:
df['home_team_received_red_card'] = df['home_team_red_cards'] > 0
df['away_team_received_red_card'] = df['away_team_red_cards'] > 0

In [94]:
# Tworzenie DataFrame z danymi dotyczącymi czerwonych kartek
red_cards_data = pd.concat([
    df[['date', 'home_team_name', 'home_team_received_red_card']].rename(columns={'home_team_name': 'team', 'home_team_received_red_card': 'received_red_card'}),
    df[['date', 'away_team_name', 'away_team_received_red_card']].rename(columns={'away_team_name': 'team', 'away_team_received_red_card': 'received_red_card'})
])

# Sortowanie danych według daty
red_cards_data.sort_values('date', inplace=True)

# Obliczanie kumulatywnych sum meczów z czerwoną kartką
red_cards_data['cumulative_games_with_red_cards'] = red_cards_data.groupby('team')['received_red_card'].cumsum()

# Usuwamy ostatni mecz, aby uzyskać dane przed bieżącym meczem
red_cards_data['games_with_red_cards_pre_game'] = red_cards_data.groupby('team')['cumulative_games_with_red_cards'].shift().fillna(0)

In [95]:
df = df.merge(red_cards_data[['date', 'team', 'games_with_red_cards_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'games_with_red_cards_pre_game': 'home_team_games_with_red_cards_pre_game'}).drop('team', axis=1)
df = df.merge(red_cards_data[['date', 'team', 'games_with_red_cards_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'games_with_red_cards_pre_game': 'away_team_games_with_red_cards_pre_game'}).drop('team', axis=1)

In [96]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game,home_team_average_fouls_per_game,away_team_average_fouls_per_game,total_fouls,home_team_average_total_fouls_per_game,away_team_average_total_fouls_per_game,home_team_average_possession_per_game,away_team_average_possession_per_game,team_a_average_xg_per_game,team_b_average_xg_per_game,home_team_failed_to_score,away_team_failed_to_score,home_team_games_without_goals_pre_game,away_team_games_without_goals_pre_game,home_team_received_red_card,away_team_received_red_card,home_team_games_with_red_cards_pre_game,away_team_games_with_red_cards_pre_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297,False,22.0,16.0,True,True,9.0,15.0,False,False,True,15.0,6.0,16.0,21.0,8.0,8.0,412.0,218.0,434.0,231.0,11.135135,5.891892,11.729730,6.243243,9.243243,8.945946,17,19.864865,20.351351,51.162162,63.756757,1.514054,1.667568,True,True,7.0,9.0,False,False,5.0,0.0
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.9

# Average total red cards

In [356]:
df['total_red_cards_in_match'] = df['home_team_red_cards'] + df['away_team_red_cards']

In [357]:
# Agregacja danych o łącznych czerwonych kartkach dla drużyny gospodarzy i gości
total_red_cards_data = pd.concat([
    df[['date', 'home_team_name', 'total_red_cards_in_match']].rename(columns={'home_team_name': 'team'}),
    df[['date', 'away_team_name', 'total_red_cards_in_match']].rename(columns={'away_team_name': 'team'})
])

# Sortowanie danych według daty
total_red_cards_data.sort_values('date', inplace=True)

In [358]:
# Obliczanie kumulatywnych sum łącznych czerwonych kartek
total_red_cards_data['cumulative_total_red_cards'] = total_red_cards_data.groupby('team')['total_red_cards_in_match'].cumsum()

# Liczenie liczby meczów
total_red_cards_data['games_played'] = total_red_cards_data.groupby('team').cumcount() + 1

# Obliczanie średniej liczby łącznych czerwonych kartek na mecz
total_red_cards_data['average_total_red_cards_per_game'] = total_red_cards_data['cumulative_total_red_cards'] / total_red_cards_data['games_played']

# Przesunięcie o jeden wiersz, aby nie uwzględniać bieżącego meczu
total_red_cards_data['average_total_red_cards_per_game_pre_game'] = total_red_cards_data.groupby('team')['average_total_red_cards_per_game'].shift().fillna(0)

In [359]:
df = df.merge(total_red_cards_data[['date', 'team', 'average_total_red_cards_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_total_red_cards_per_game_pre_game': 'home_team_average_total_red_cards_per_game'}).drop('team', axis=1)
df = df.merge(total_red_cards_data[['date', 'team', 'average_total_red_cards_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_total_red_cards_per_game_pre_game': 'away_team_average_total_red_cards_per_game'}).drop('team', axis=1)

In [360]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_goals_scored_home_team_before_game,cumulative_goals_scored_away_team_before_game,cumulative_goals_conceded_home_team_before_game,cumulative_goals_conceded_away_team_before_game,home_goals_scored,away_goals_scored,home_goals_conceded,away_goals_conceded,home_team_average_goals_scored_per_game,away_team_average_goals_scored_per_game,home_team_average_goals_conceded_per_game,away_team_average_goals_conceded_per_game,home_team_second_half_goals,away_team_second_half_goals,home_team_average_first_half_goals,home_team_average_second_half_goals,away_team_average_first_half_goals,away_team_average_second_half_goals,home_goals_conceded_first_half,away_goals_conceded_first_half,home_goals_conceded_second_half,away_goals_conceded_second_half,home_team_average_first_half_goals_conceded,home_team_average_second_half_goals_conceded,away_team_average_first_half_goals_conceded,away_team_average_second_half_goals_conceded,home_first_half_goals,home_second_half_goals,away_first_half_goals,away_second_half_goals,home_team_average_first_half_goals_total,home_team_average_second_half_goals_total,away_team_average_first_half_goals_total,away_team_average_second_half_goals_total,total_red_cards_in_game,cumulative_total_red_cards,total_games,average_total_red_cards_per_game,average_total_red_cards_per_game_pre_game_x,home_team_average_red_cards_per_game,away_team_average_red_cards_per_game,total_red_cards_in_match,average_total_red_cards_per_game_pre_game_y,away_team_average_total_red_cards_per_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,51,63,48,39,0,0,0,0,1.378378,1.702703,1.297297,1.054054,0,0,0.405405,0.972973,0.729730,0.972973,0,0,0,0,0.729730,0.972973,0.459459,0.972973,0,0,0,0,0.405405,0.972973,0.729730,0.972973,0,0,6,0.0,0.0,0.135135,0.000000,0,0.297297,0.000000
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,87,47,22,44,2,0,0,2,2.351351,1.270270,0.594595,1.189189,1,0,1.000000,1.351351,0.459459,0.810811,0,

# Average red cards per team before game

In [350]:
import pandas as pd

# Tworzenie DataFrame z danymi dotyczącymi czerwonych kartek
red_cards_data = pd.concat([
    df[['date', 'home_team_name', 'home_team_red_cards']].rename(columns={'home_team_name': 'team', 'home_team_red_cards': 'red_cards'}),
    df[['date', 'away_team_name', 'away_team_red_cards']].rename(columns={'away_team_name': 'team', 'away_team_red_cards': 'red_cards'})
])

# Sortowanie danych według daty
red_cards_data.sort_values('date', inplace=True)

In [351]:
# Obliczanie kumulatywnych sum czerwonych kartek
red_cards_data['cumulative_red_cards'] = red_cards_data.groupby('team')['red_cards'].cumsum()

# Liczenie liczby meczów
red_cards_data['games_played'] = red_cards_data.groupby('team').cumcount() + 1

# Obliczanie średniej liczby czerwonych kartek na mecz
red_cards_data['average_red_cards_per_game'] = red_cards_data['cumulative_red_cards'] / red_cards_data['games_played']

In [353]:
red_cards_data['average_red_cards_per_game_pre_game'] = red_cards_data.groupby('team')['average_red_cards_per_game'].shift().fillna(0)

In [354]:
df = df.merge(red_cards_data[['date', 'team', 'average_red_cards_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_red_cards_per_game_pre_game': 'home_team_average_red_cards_per_game'}).drop('team', axis=1)
df = df.merge(red_cards_data[['date', 'team', 'average_red_cards_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_red_cards_per_game_pre_game': 'away_team_average_red_cards_per_game'}).drop('team', axis=1)

In [355]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_goals_scored_home_team_before_game,cumulative_goals_scored_away_team_before_game,cumulative_goals_conceded_home_team_before_game,cumulative_goals_conceded_away_team_before_game,home_goals_scored,away_goals_scored,home_goals_conceded,away_goals_conceded,home_team_average_goals_scored_per_game,away_team_average_goals_scored_per_game,home_team_average_goals_conceded_per_game,away_team_average_goals_conceded_per_game,home_team_second_half_goals,away_team_second_half_goals,home_team_average_first_half_goals,home_team_average_second_half_goals,away_team_average_first_half_goals,away_team_average_second_half_goals,home_goals_conceded_first_half,away_goals_conceded_first_half,home_goals_conceded_second_half,away_goals_conceded_second_half,home_team_average_first_half_goals_conceded,home_team_average_second_half_goals_conceded,away_team_average_first_half_goals_conceded,away_team_average_second_half_goals_conceded,home_first_half_goals,home_second_half_goals,away_first_half_goals,away_second_half_goals,home_team_average_first_half_goals_total,home_team_average_second_half_goals_total,away_team_average_first_half_goals_total,away_team_average_second_half_goals_total,total_red_cards_in_game,cumulative_total_red_cards,total_games,average_total_red_cards_per_game,average_total_red_cards_per_game_pre_game,home_team_average_red_cards_per_game,away_team_average_red_cards_per_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,51,63,48,39,0,0,0,0,1.378378,1.702703,1.297297,1.054054,0,0,0.405405,0.972973,0.729730,0.972973,0,0,0,0,0.729730,0.972973,0.459459,0.972973,0,0,0,0,0.405405,0.972973,0.729730,0.972973,0,0,6,0.0,0.0,0.135135,0.000000
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,87,47,22,44,2,0,0,2,2.351351,1.270270,0.594595,1.189189,1,0,1.000000,1.351351,0.459459,0.810811,0,1,1,0,0.270270,1.351351,0.567568,0.810811,1,1,0,0,1.000000,1.351351,0.459459,0.810811,0,0,7,0.0,0.0,0.054054,0.027027
377,74457,Manche

# Average shots before the game

In [59]:
import pandas as pd

# Załóżmy, że `df` to Twój DataFrame z danymi
# Tworzenie DataFrame z danymi dotyczącymi strzałów
shots_data = pd.concat([
    df[['date', 'home_team_name', 'home_team_shots', 'home_team_shots_on_target']].rename(columns={'home_team_name': 'team', 'home_team_shots': 'shots', 'home_team_shots_on_target': 'shots_on_target'}),
    df[['date', 'away_team_name', 'away_team_shots', 'away_team_shots_on_target']].rename(columns={'away_team_name': 'team', 'away_team_shots': 'shots', 'away_team_shots_on_target': 'shots_on_target'})
])

# Sortowanie danych według daty
shots_data.sort_values('date', inplace=True)

In [60]:
# Obliczanie kumulatywnych sum strzałów i strzałów celnym
shots_data['cumulative_shots'] = shots_data.groupby('team')['shots'].cumsum()
shots_data['cumulative_shots_on_target'] = shots_data.groupby('team')['shots_on_target'].cumsum()

# Liczenie liczby meczów
shots_data['games_played'] = shots_data.groupby('team').cumcount() + 1

# Obliczanie średnich kumulatywnych na mecz
shots_data['average_shots_per_game'] = shots_data['cumulative_shots'] / shots_data['games_played']
shots_data['average_shots_on_target_per_game'] = shots_data['cumulative_shots_on_target'] / shots_data['games_played']

In [61]:
shots_data['average_shots_per_game_pre_game'] = shots_data.groupby('team')['average_shots_per_game'].shift().fillna(0)
shots_data['average_shots_on_target_per_game_pre_game'] = shots_data.groupby('team')['average_shots_on_target_per_game'].shift().fillna(0)

In [62]:
df = df.merge(shots_data[['date', 'team', 'average_shots_per_game_pre_game', 'average_shots_on_target_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_shots_per_game_pre_game': 'home_team_average_shots_per_game', 'average_shots_on_target_per_game_pre_game': 'home_team_average_shots_on_target_per_game'}).drop('team', axis=1)
df = df.merge(shots_data[['date', 'team', 'average_shots_per_game_pre_game', 'average_shots_on_target_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_shots_per_game_pre_game': 'away_team_average_shots_per_game', 'average_shots_on_target_per_game_pre_game': 'away_team_average_shots_on_target_per_game'}).drop('team', axis=1)

In [63]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297,False,22.0,16.0,True,True,9.0,15.0,False,False,True,15.0,6.0,16.0,21.0,8.0,8.0,412.0,218.0,434.0,231.0,11.135135,5.891892,11.729730,6.243243
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,22,44,10.027027,10.270270,6.621622,5.243243,2.297297,3.945946,1.054054,1.891892,False,17.0,19.0,True,False,20.0,9.0,True,False,False,29.0,7.0,1.0,16.0,9.0,12.0,461.0,258.0,390.0,185.0,12.459459,6.972973,10.540541,5.000000
377,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,1.89,0.74,0,2,2,1,0,1,11,2,3,0,3,0,1,2,1,2,21,11,11,5,10,6,9,6,73,27,1.67,1.10,2.67,1.31,2.72,56,78,56,31,20,25,64,53,78,9.77,3.50,1.28,6.2,10.0,1.12,1.41,2.00,3.60,1.74,2.05,Old Trafford (Manchester),premier league

# Average fouls per team before the game

In [66]:
import pandas as pd

# Tworzenie DataFrame z danymi dotyczącymi fauli
fouls_data = pd.concat([
    df[['date', 'home_team_name', 'home_team_fouls']].rename(columns={'home_team_name': 'team', 'home_team_fouls': 'fouls'}),
    df[['date', 'away_team_name', 'away_team_fouls']].rename(columns={'away_team_name': 'team', 'away_team_fouls': 'fouls'})
])

# Sortowanie danych według daty
fouls_data.sort_values('date', inplace=True)

In [67]:
# Obliczanie kumulatywnych sum fauli
fouls_data['cumulative_fouls'] = fouls_data.groupby('team')['fouls'].cumsum()

# Liczenie liczby meczów
fouls_data['games_played'] = fouls_data.groupby('team').cumcount() + 1

# Obliczanie średnich kumulatywnych na mecz
fouls_data['average_fouls_per_game'] = fouls_data['cumulative_fouls'] / fouls_data['games_played']

In [68]:
fouls_data['average_fouls_per_game_pre_game'] = fouls_data.groupby('team')['average_fouls_per_game'].shift().fillna(0)

In [69]:
df = df.merge(fouls_data[['date', 'team', 'average_fouls_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_fouls_per_game_pre_game': 'home_team_average_fouls_per_game'}).drop('team', axis=1)
df = df.merge(fouls_data[['date', 'team', 'average_fouls_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_fouls_per_game_pre_game': 'away_team_average_fouls_per_game'}).drop('team', axis=1)

In [70]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game,home_team_average_fouls_per_game,away_team_average_fouls_per_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297,False,22.0,16.0,True,True,9.0,15.0,False,False,True,15.0,6.0,16.0,21.0,8.0,8.0,412.0,218.0,434.0,231.0,11.135135,5.891892,11.729730,6.243243,9.243243,8.945946
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,22,44,10.027027,10.270270,6.621622,5.243243,2.297297,3.945946,1.054054,1.891892,False,17.0,19.0,True,False,20.0,9.0,True,False,False,29.0,7.0,1.0,16.0,9.0,12.0,461.0,258.0,390.0,185.0,12.459459,6.972973,10.540541,5.000000,8.432432,10.756757
377,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,1.89,0.74,0,2,2,1,0,1,11,2,3,0,3,0,1,2,1,2,21,11,11,5,10,6,9,6,73,27,1.67,1.10,2.67,1.31,2.72,56,78,56,31,20,25,

# Average fouls overall before game

In [71]:
df['total_fouls'] = df['home_team_fouls'] + df['away_team_fouls']

In [72]:
# Tworzenie DataFrame z danymi dotyczącymi łącznych fauli dla obu drużyn w meczu
fouls_data = pd.concat([
    df[['date', 'home_team_name', 'total_fouls']].rename(columns={'home_team_name': 'team'}),
    df[['date', 'away_team_name', 'total_fouls']].rename(columns={'away_team_name': 'team'})
])

# Sortowanie danych według daty
fouls_data.sort_values('date', inplace=True)

# Obliczanie kumulatywnej sumy łącznych fauli
fouls_data['cumulative_total_fouls'] = fouls_data.groupby('team')['total_fouls'].cumsum()

# Liczenie liczby meczów
fouls_data['games_played'] = fouls_data.groupby('team').cumcount() + 1

# Obliczanie średniej łącznej liczby fauli na mecz
fouls_data['average_total_fouls_per_game'] = fouls_data['cumulative_total_fouls'] / fouls_data['games_played']

# Usuwamy ostatni mecz, aby uzyskać dane przed bieżącym meczem
fouls_data['average_total_fouls_per_game_pre_game'] = fouls_data.groupby('team')['average_total_fouls_per_game'].shift().fillna(0)

In [73]:
df = df.merge(fouls_data[['date', 'team', 'average_total_fouls_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_total_fouls_per_game_pre_game': 'home_team_average_total_fouls_per_game'}).drop('team', axis=1)
df = df.merge(fouls_data[['date', 'team', 'average_total_fouls_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_total_fouls_per_game_pre_game': 'away_team_average_total_fouls_per_game'}).drop('team', axis=1)

In [74]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game,home_team_average_fouls_per_game,away_team_average_fouls_per_game,total_fouls,home_team_average_total_fouls_per_game,away_team_average_total_fouls_per_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297,False,22.0,16.0,True,True,9.0,15.0,False,False,True,15.0,6.0,16.0,21.0,8.0,8.0,412.0,218.0,434.0,231.0,11.135135,5.891892,11.729730,6.243243,9.243243,8.945946,17,19.864865,20.351351
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,22,44,10.027027,10.270270,6.621622,5.243243,2.297297,3.945946,1.054054,1.891892,False,17.0,19.0,True,False,20.0,9.0,True,False,False,29.0,7.0,1.0,16.0,9.0,12.0,461.0,258.0,390.0,185.0,12.459459,6.972973,10.540541,5.000000,8.432432,10.756757,14,18.054054,19.702703
377,74457,Manchester United,Cardiff City,Jon

# Average ball possession before game

In [76]:
# Tworzenie DataFrame z danymi dotyczącymi posiadania piłki
possession_data = pd.concat([
    df[['date', 'home_team_name', 'home_team_possession']].rename(columns={'home_team_name': 'team', 'home_team_possession': 'possession'}),
    df[['date', 'away_team_name', 'away_team_possession']].rename(columns={'away_team_name': 'team', 'away_team_possession': 'possession'})
])

# Sortowanie danych według daty
possession_data.sort_values('date', inplace=True)

In [77]:
# Obliczanie kumulatywnych sum posiadania piłki
possession_data['cumulative_possession'] = possession_data.groupby('team')['possession'].cumsum()

# Liczenie liczby meczów
possession_data['games_played'] = possession_data.groupby('team').cumcount() + 1

# Obliczanie średniego posiadania piłki na mecz
possession_data['average_possession_per_game'] = possession_data['cumulative_possession'] / possession_data['games_played']

In [78]:
possession_data['average_possession_per_game_pre_game'] = possession_data.groupby('team')['average_possession_per_game'].shift().fillna(0)

In [79]:
df = df.merge(possession_data[['date', 'team', 'average_possession_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_possession_per_game_pre_game': 'home_team_average_possession_per_game'}).drop('team', axis=1)
df = df.merge(possession_data[['date', 'team', 'average_possession_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_possession_per_game_pre_game': 'away_team_average_possession_per_game'}).drop('team', axis=1)

In [80]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game,home_team_average_fouls_per_game,away_team_average_fouls_per_game,total_fouls,home_team_average_total_fouls_per_game,away_team_average_total_fouls_per_game,home_team_average_possession_per_game,away_team_average_possession_per_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297,False,22.0,16.0,True,True,9.0,15.0,False,False,True,15.0,6.0,16.0,21.0,8.0,8.0,412.0,218.0,434.0,231.0,11.135135,5.891892,11.729730,6.243243,9.243243,8.945946,17,19.864865,20.351351,51.162162,63.756757
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,22,44,10.027027,10.270270,6.621622,5.243243,2.297297,3.945946,1.054054,1.891892,False,17.0,19.0,True,False,20.0,9.0,True,False,False,29.0,7.0,1.0,16.0,9.0,12.0,461.0,258.0,390.0,185.0,12.459459,6.972973,10.540541

# Average xg before game

In [82]:
df.head()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game,home_team_average_fouls_per_game,away_team_average_fouls_per_game,total_fouls,home_team_average_total_fouls_per_game,away_team_average_total_fouls_per_game,home_team_average_possession_per_game,away_team_average_possession_per_game
0,74439,Manchester United,Leicester City,Andre Marriner,1,0.0,0.0,1.89,1.32,2,1,3,1,1,0,2,5,2,0,1,0,0,2,0,1,8,8,7,5,1,3,11,8,46,54,0.0,0.0,1.25,1.46,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.37,4.98,9.81,1.33,2.00,3.60,7.50,2.05,1.69,Old Trafford (Manchester),premier league,2018-08-10,19:00:00,7,3,0,3,16,12,4,19,1,18/19,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0,0.0,False,False,0.0,0.0,True,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,0.0,0.0,0.0,0.0
1,51749,Newcastle United,Tottenham Hotspur,Martin Atkinson,1,0.0,0.0,1.32,1.74,1,2,3,3,1,2,3,5,2,0,2,0,0,2,1,1,11,13,3,6,8,7,11,12,40,60,0.0,0.0,1.18,1.58,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,4.51,3.77,1.84,1.31,1.95,3.50,5.75,1.83,1.87,St. James' Park (Newcastle upon Tyne),premier league,2018-08-11,11:30:00,8,4,0,4,24,9,15,23,2,18/19,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0,0.0,False,False,0.0,0.0,False,True,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23,0.0,0.0,0.0,0.0
2,10353,AFC Bournemouth,Cardiff City,Kevin Friend,1,0.0,0.0,1.53,0.74,2,0,2,1,1,0,7,4,1,0,1,0,1,0,1,0,11,7,5,2,6,5,11,9,62,38,0.0,0.0,1.50,0.89,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.03,

In [84]:
# Tworzenie DataFrame z danymi dotyczącymi xG
xg_data = pd.concat([
    df[['date', 'home_team_name', 'team_a_xg']].rename(columns={'home_team_name': 'team', 'team_a_xg': 'xg'}),
    df[['date', 'away_team_name', 'team_b_xg']].rename(columns={'away_team_name': 'team', 'team_b_xg': 'xg'})
])

# Sortowanie danych według daty
xg_data.sort_values('date', inplace=True)

In [85]:
# Obliczanie kumulatywnych sum xG
xg_data['cumulative_xg'] = xg_data.groupby('team')['xg'].cumsum()

# Liczenie liczby meczów
xg_data['games_played'] = xg_data.groupby('team').cumcount() + 1

# Obliczanie średniej xG na mecz
xg_data['average_xg_per_game'] = xg_data['cumulative_xg'] / xg_data['games_played']


In [86]:
xg_data['average_xg_per_game_pre_game'] = xg_data.groupby('team')['average_xg_per_game'].shift().fillna(0)


In [87]:
df = df.merge(xg_data[['date', 'team', 'average_xg_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_xg_per_game_pre_game': 'team_a_average_xg_per_game'}).drop('team', axis=1)
df = df.merge(xg_data[['date', 'team', 'average_xg_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_xg_per_game_pre_game': 'team_b_average_xg_per_game'}).drop('team', axis=1)


In [88]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game,home_team_average_fouls_per_game,away_team_average_fouls_per_game,total_fouls,home_team_average_total_fouls_per_game,away_team_average_total_fouls_per_game,home_team_average_possession_per_game,away_team_average_possession_per_game,team_a_average_xg_per_game,team_b_average_xg_per_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297,False,22.0,16.0,True,True,9.0,15.0,False,False,True,15.0,6.0,16.0,21.0,8.0,8.0,412.0,218.0,434.0,231.0,11.135135,5.891892,11.729730,6.243243,9.243243,8.945946,17,19.864865,20.351351,51.162162,63.756757,1.514054,1.667568
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,22,44,10.027027,10.270270,6.621622,5.243243,2.297297,3.945946,1.054054,1.891892,False,17.0,19.0,True,False,20.0,9.0,True,False,False,29.0,7.

# No goals scored before game

In [89]:
import pandas as pd

df['home_team_failed_to_score'] = df['home_team_goal_count'] == 0
df['away_team_failed_to_score'] = df['away_team_goal_count'] == 0

In [90]:
# Tworzenie DataFrame z danymi dotyczącymi braku strzelonych bramek
no_goals_data = pd.concat([
    df[['date', 'home_team_name', 'home_team_failed_to_score']].rename(columns={'home_team_name': 'team', 'home_team_failed_to_score': 'failed_to_score'}),
    df[['date', 'away_team_name', 'away_team_failed_to_score']].rename(columns={'away_team_name': 'team', 'away_team_failed_to_score': 'failed_to_score'})
])

# Sortowanie danych według daty
no_goals_data.sort_values('date', inplace=True)

# Obliczanie kumulatywnych sum meczów bez strzelonych bramek
no_goals_data['cumulative_games_without_goals'] = no_goals_data.groupby('team')['failed_to_score'].cumsum()

# Usuwamy ostatni mecz, aby uzyskać dane przed bieżącym meczem
no_goals_data['games_without_goals_pre_game'] = no_goals_data.groupby('team')['cumulative_games_without_goals'].shift().fillna(0)

In [91]:
df = df.merge(no_goals_data[['date', 'team', 'games_without_goals_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'games_without_goals_pre_game': 'home_team_games_without_goals_pre_game'}).drop('team', axis=1)
df = df.merge(no_goals_data[['date', 'team', 'games_without_goals_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'games_without_goals_pre_game': 'away_team_games_without_goals_pre_game'}).drop('team', axis=1)

In [92]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game,home_team_average_fouls_per_game,away_team_average_fouls_per_game,total_fouls,home_team_average_total_fouls_per_game,away_team_average_total_fouls_per_game,home_team_average_possession_per_game,away_team_average_possession_per_game,team_a_average_xg_per_game,team_b_average_xg_per_game,home_team_failed_to_score,away_team_failed_to_score,home_team_games_without_goals_pre_game,away_team_games_without_goals_pre_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297,False,22.0,16.0,True,True,9.0,15.0,False,False,True,15.0,6.0,16.0,21.0,8.0,8.0,412.0,218.0,434.0,231.0,11.135135,5.891892,11.729730,6.243243,9.243243,8.945946,17,19.864865,20.351351,51.162162,63.756757,1.514054,1.667568,True,True,7.0,9.0
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,

# BTTS before game

In [29]:
df['both_teams_scored'] = (df['home_team_goal_count'] > 0) & (df['away_team_goal_count'] > 0)

In [30]:
df['both_teams_scored'].value_counts()

both_teams_scored
True     195
False    185
Name: count, dtype: int64

In [31]:
# Tworzenie DataFrame z danymi dotyczącymi, czy obie drużyny strzeliły
scored_data = pd.concat([
    df[['date', 'home_team_name', 'both_teams_scored']].rename(columns={'home_team_name': 'team'}),
    df[['date', 'away_team_name', 'both_teams_scored']].rename(columns={'away_team_name': 'team'})
])

# Sortowanie danych według daty
scored_data.sort_values('date', inplace=True)

# Obliczanie kumulatywnej sumy meczów, gdzie obie drużyny strzeliły
scored_data['cumulative_both_scored'] = scored_data.groupby('team')['both_teams_scored'].cumsum()

# Usuwamy ostatni mecz, aby uzyskać dane przed bieżącym meczem
scored_data['cumulative_both_scored_pre_game'] = scored_data.groupby('team')['cumulative_both_scored'].shift().fillna(0)

In [32]:
df = df.merge(scored_data[['date', 'team', 'cumulative_both_scored_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_both_scored_pre_game': 'home_team_cumulative_both_scored_pre_game'}).drop('team', axis=1)
df = df.merge(scored_data[['date', 'team', 'cumulative_both_scored_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_both_scored_pre_game': 'away_team_cumulative_both_scored_pre_game'}).drop('team', axis=1)

In [33]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297,False,22.0,16.0
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,22,44,10.027027,10.270270,6.621622,5.243243,2.297297,3.945946,1.054054,1.891892,False,17.0,19.0
377,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,1.89,0.74,0,2,2,1,0,1,11,2,3,0,3,0,1,2,1,2,21,11,11,5,10,6,9,6,73,27,1.67,1.10,2.67,1.31,2.72,56,78,56,31,20,25,64,53,78,9.77,3.50,1.28,6.2,10.0,1.12,1.41,2.00,3.60,1.74,2.05,Old Trafford (Manchester),premier league,2019-05-12,14:00:00,13,6,0,6,32,16,16,15,2,18/19,52,69,10.081081,11.486486,5.108108,4.486486,3.675676,2.945946,1.945946,1.702703,False,26.0,15.0
378,30367,Southampton,Huddersfield Town,Lee Probert,38,1.22,0.33,1.21,0.37,1,1,2,1,1,0,4,3,0,0,1,0,0,0,0,1,7,8,4,4,3,4,8,6,53,47,1.46,1.11,1.15,1.04,3.06,59,84,61,39,17,45,67,53,78,9.50,4.00,1.41,4.9,7.5,1.19,1.61,2.50,4.75,1.80,2.00,St. Mary's Stadium (Southampton- Hampshire),premier league,2019-05-12,14:00:00,7,1,0,1,15,8,7,14,0,18/19,64,75,10.837838,8.783784,5.135135,4.297297,3.621622,3.270270,1.972973,1.486486,True,24.0,15.0
379,20067,Watford,West Ham United,Chris Kavanagh,38,1.50,1.00,1.42,1.11,1,4,5,2,0,2,7,2,1,1,0,0,0,2,0,0,11,15,9,10,2,5,10,10,48,52,1.30,1.18,1.81,2.01,2.56,50,83,53,17,6,28,72,42,86,9.00,4.11,2.20,3.7,3.1,1.16,1.54,2.35,4.00,1.48,2.55,Vicarage Road (Watford),premier league,2019-05-12,14:00:00,9,1,1,2,26,19,7,20,2,18/19,55,54,10.216216,10.432432,

# Clean sheets before the game

In [35]:
df['home_clean_sheet'] = df['away_team_goal_count'] == 0
df['away_clean_sheet'] = df['home_team_goal_count'] == 0

In [36]:
# Tworzenie DataFrame z danymi dotyczącymi czystych kont
clean_sheets_data = pd.concat([
    df[['date', 'home_team_name', 'home_clean_sheet']].rename(columns={'home_team_name': 'team', 'home_clean_sheet': 'clean_sheet'}),
    df[['date', 'away_team_name', 'away_clean_sheet']].rename(columns={'away_team_name': 'team', 'away_clean_sheet': 'clean_sheet'})
])

# Sortowanie danych według daty
clean_sheets_data.sort_values('date', inplace=True)

# Obliczanie kumulatywnej sumy czystych kont
clean_sheets_data['cumulative_clean_sheets'] = clean_sheets_data.groupby('team')['clean_sheet'].cumsum()

# Usuwamy ostatni mecz, aby uzyskać dane przed bieżącym meczem
clean_sheets_data['cumulative_clean_sheets_pre_game'] = clean_sheets_data.groupby('team')['cumulative_clean_sheets'].shift().fillna(0)


In [38]:
df = df.merge(clean_sheets_data[['date', 'team', 'cumulative_clean_sheets_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_clean_sheets_pre_game': 'home_team_cumulative_clean_sheets_pre_game'}).drop('team', axis=1)
df = df.merge(clean_sheets_data[['date', 'team', 'cumulative_clean_sheets_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_clean_sheets_pre_game': 'away_team_cumulative_clean_sheets_pre_game'}).drop('team', axis=1)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 91 columns):
 #   Column                                                Non-Null Count  Dtype         
---  ------                                                --------------  -----         
 0   attendance                                            380 non-null    int64         
 1   home_team_name                                        380 non-null    object        
 2   away_team_name                                        380 non-null    object        
 3   referee                                               380 non-null    object        
 4   Game Week                                             380 non-null    int64         
 5   Pre-Match PPG (Home)                                  380 non-null    float64       
 6   Pre-Match PPG (Away)                                  380 non-null    float64       
 7   home_ppg                                              380 non-null    float64   

# Wins, losses and draws before the game

In [40]:
df['home_win'] = df['home_team_goal_count'] > df['away_team_goal_count']
df['away_win'] = df['home_team_goal_count'] < df['away_team_goal_count']
df['draw'] = df['home_team_goal_count'] == df['away_team_goal_count']

In [41]:
# Tworzenie DataFrame z danymi dotyczącymi wyników meczów
results_data = pd.concat([
    df[['date', 'home_team_name', 'home_win', 'draw', 'away_win']].rename(columns={'home_team_name': 'team', 'home_win': 'win', 'away_win': 'loss'}),
    df[['date', 'away_team_name', 'away_win', 'draw', 'home_win']].rename(columns={'away_team_name': 'team', 'away_win': 'win', 'home_win': 'loss'})
])

# Sortowanie danych według daty
results_data.sort_values('date', inplace=True)

# Obliczanie kumulatywnych sum zwycięstw, remisów i porażek
results_data['cumulative_wins'] = results_data.groupby('team')['win'].cumsum()
results_data['cumulative_draws'] = results_data.groupby('team')['draw'].cumsum()
results_data['cumulative_losses'] = results_data.groupby('team')['loss'].cumsum()

# Usuwamy ostatni mecz, aby uzyskać dane przed bieżącym meczem
results_data['cumulative_wins_pre_game'] = results_data.groupby('team')['cumulative_wins'].shift().fillna(0)
results_data['cumulative_draws_pre_game'] = results_data.groupby('team')['cumulative_draws'].shift().fillna(0)
results_data['cumulative_losses_pre_game'] = results_data.groupby('team')['cumulative_losses'].shift().fillna(0)

In [42]:
df = df.merge(results_data[['date', 'team', 'cumulative_wins_pre_game', 'cumulative_draws_pre_game', 'cumulative_losses_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_wins_pre_game': 'home_team_cumulative_wins_pre_game', 'cumulative_draws_pre_game': 'home_team_cumulative_draws_pre_game', 'cumulative_losses_pre_game': 'home_team_cumulative_losses_pre_game'}).drop('team', axis=1)
df = df.merge(results_data[['date', 'team', 'cumulative_wins_pre_game', 'cumulative_draws_pre_game', 'cumulative_losses_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_wins_pre_game': 'away_team_cumulative_wins_pre_game', 'cumulative_draws_pre_game': 'away_team_cumulative_draws_pre_game', 'cumulative_losses_pre_game': 'away_team_cumulative_losses_pre_game'}).drop('team', axis=1)

In [75]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game,home_team_average_fouls_per_game,away_team_average_fouls_per_game,total_fouls,home_team_average_total_fouls_per_game,away_team_average_total_fouls_per_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297,False,22.0,16.0,True,True,9.0,15.0,False,False,True,15.0,6.0,16.0,21.0,8.0,8.0,412.0,218.0,434.0,231.0,11.135135,5.891892,11.729730,6.243243,9.243243,8.945946,17,19.864865,20.351351
376,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,2.79,1.21,2,0,2,1,1,0,4,1,0,0,2,0,0,0,0,2,11,7,6,3,5,4,3,11,59,41,1.96,1.27,1.40,0.92,2.92,50,81,53,34,17,39,75,42,86,11.89,3.00,1.32,5.8,9.0,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),premier league,2019-05-12,14:00:00,5,2,0,2,18,9,9,14,1,18/19,22,44,10.027027,10.270270,6.621622,5.243243,2.297297,3.945946,1.054054,1.891892,False,17.0,19.0,True,False,20.0,9.0,True,False,False,29.0,7.0,1.0,16.0,9.0,12.0,461.0,258.0,390.0,185.0,12.459459,6.972973,10.540541,5.000000,8.432432,10.756757,14,18.054054,19.702703
377,74457,Manchester United,Cardiff City,Jon

# Points per game before the game

In [254]:
df['home_win'] = df['home_team_goal_count'] > df['away_team_goal_count']
df['away_win'] = df['home_team_goal_count'] < df['away_team_goal_count']
df['draw'] = df['home_team_goal_count'] == df['away_team_goal_count']

In [255]:
df['home_points'] = df['home_win'] * 3 + df['draw'] * 1
df['away_points'] = df['away_win'] * 3 + df['draw'] * 1

In [256]:
points_data = pd.concat([
    df[['date', 'home_team_name', 'home_points']].rename(columns={'home_team_name': 'team', 'home_points': 'points'}),
    df[['date', 'away_team_name', 'away_points']].rename(columns={'away_team_name': 'team', 'away_points': 'points'})
])

# Sortowanie danych według daty
points_data.sort_values('date', inplace=True)

In [257]:
# Obliczanie kumulatywnych sum punktów
points_data['cumulative_points'] = points_data.groupby('team')['points'].cumsum()

# Liczenie liczby meczów
points_data['games_played'] = points_data.groupby('team').cumcount() + 1

# Obliczanie średniej liczby punktów na mecz
points_data['average_points_per_game'] = points_data['cumulative_points'] / points_data['games_played']

# Usuwamy ostatni mecz, aby uzyskać dane przed bieżącym meczem
points_data['average_points_per_game_pre_game'] = points_data.groupby('team')['average_points_per_game'].shift().fillna(0)

In [258]:
df = df.merge(points_data[['date', 'team', 'average_points_per_game_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_points_per_game_pre_game': 'home_team_average_points_per_game'}).drop('team', axis=1)
df = df.merge(points_data[['date', 'team', 'average_points_per_game_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_points_per_game_pre_game': 'away_team_average_points_per_game'}).drop('team', axis=1)

In [259]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game,home_team_average_fouls_per_game,away_team_average_fouls_per_game,total_fouls,home_team_average_total_fouls_per_game,away_team_average_total_fouls_per_game,home_team_average_possession_per_game,away_team_average_possession_per_game,team_a_average_xg_per_game,team_b_average_xg_per_game,home_team_failed_to_score,away_team_failed_to_score,home_team_games_without_goals_pre_game,away_team_games_without_goals_pre_game,home_team_received_red_card,away_team_received_red_card,home_team_games_with_red_cards_pre_game,away_team_games_with_red_cards_pre_game,home_team_average_red_cards_per_game,away_team_average_red_cards_per_game,home_points,away_points,home_team_average_points_per_game,away_team_average_points_per_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297,False,22.0,16.0,True,True,9.0,15.0,False,False,True,15.0,6.0,16.0,21.0,8.0,8.0,412.0,218.0,434.0,231.0,11.135135,5.891892,11.729730,6.243243,9.243243,8.945946,17,19.864865,20.351351,51.162162,63.756757,1.514054,1.667568,True,True,7.0,9.0,False,False,5.0,

# Specific variables to create

In [47]:
df_vars_to_create = pd.read_csv('england-premier-league-teams2-2018-to-2019-stats (1).csv')

In [48]:
df_vars_to_create.head()

,team_name,common_name,season,country,average_attendance_overall,average_attendance_home,average_attendance_away,minutes_per_goal_scored_overall,minutes_per_goal_scored_home,minutes_per_goal_scored_away,minutes_per_goal_conceded_overall,minutes_per_goal_conceded_home,minutes_per_goal_conceded_away,highest_goals_scored_home,highest_goals_scored_away,highest_goals_conceded_home,highest_goals_conceded_away,half_time_points_overall,half_time_points_home,half_time_points_away,corners_recorded_matches_num_overall,corners_recorded_matches_num_home,cornersRecorded_matches_away,corners_total_per_match_overall,corners_total_per_match_home,corners_total_per_match_away,corners_against_overall,corners_against_home,corners_against_away,corners_against_per_match_overall,corners_against_per_match_home,corners_against_per_match_away,over65_corners_num_overall,over75_corners_num_overall,over85_corners_num_overall,over95_corners_num_overall,over105_corners_num_overall,over115_corners_num_overall,over125_corners_num_overall,over135_corners_num_overall,over145_corners_num_overall,over65_corners_num_home,over75_corners_num_home,over85_corners_num_home,over95_corners_num_home,over105_corners_num_home,over115_corners_num_home,over125_corners_num_home,over135_corners_num_home,over145_corners_num_home,over65_corners_percentage_home,over75_corners_percentage_home,over85_corners_percentage_home,over95_corners_percentage_home,over105_corners_percentage_home,over115_corners_percentage_home,over125_corners_percentage_home,over135_corners_percentage_home,over145_corners_percentage_home,over65_corners_num_away,over75_corners_num_away,over85_corners_num_away,over95_corners_num_away,over105_corners_num_away,over115_corners_num_away,over125_corners_num_away,over135_corners_num_away,over145_corners_num_away,over65_corners_percentage_away,over75_corners_percentage_away,over85_corners_percentage_away,over95_corners_percentage_away,over105_corners_percentage_away,over115_corners_percentage_away,over125_corners_percentage_away,over135_corners_percentage_away,over145_corners_percentage_away,over25_corners_for_overall,over35_corners_for_overall,over45_corners_for_overall,over55_corners_for_overall,over65_corners_for_overall,over75_corners_for_overall,over85_corners_for_overall,over25_corners_for_percentage_overall,over35_corners_for_percentage_overall,over45_corners_for_percentage_overall,over55_corners_for_percentage_overall,over65_corners_for_percentage_overall,over75_corners_for_percentage_overall,over85_corners_for_percentage_overall,over25_corners_for_home,over35_corners_for_home,over45_corners_for_home,over55_corners_for_home,over65_corners_for_home,over75_corners_for_home,over85_corners_for_home,over25_corners_for_percentage_home,over35_corners_for_percentage_home,over45_corners_for_percentage_home,over55_corners_for_percentage_home,over65_corners_for_percentage_home,over75_corners_for_percentage_home,over85_corners_for_percentage_home,over25_corners_for_away,over35_corners_for_away,over45_corners_for_away,over55_corners_for_away,over65_corners_for_away,over75_corners_for_away,over85_corners_for_away,over25_corners_for_percentage_away,over35_corners_for_percentage_away,over45_corners_for_percentage_away,over55_corners_for_percentage_away,over65_corners_for_percentage_away,over75_corners_for_percentage_away,over85_corners_for_percentage_away,over25_corners_against_overall,over35_corners_against_overall,over45_corners_against_overall,over55_corners_against_overall,over65_corners_against_overall,over75_corners_against_overall,over85_corners_against_overall,over25_corners_against_percentage_overall,over35_corners_against_percentage_overall,over45_corners_against_percentage_overall,over55_corners_against_percentage_overall,over65_corners_against_percentage_overall,over75_corners_against_percentage_overall,over85_corners_against_percentage_overall,over25_corners_against_home,over35_corners_against_home,over45_corners_against_home,over55_corners_against_home,over65_cor

In [54]:
df_vars_to_create[[col for col in df_vars_to_create.columns if not col.startswith('over')]].head()

,team_name,common_name,season,country,average_attendance_overall,average_attendance_home,average_attendance_away,minutes_per_goal_scored_overall,minutes_per_goal_scored_home,minutes_per_goal_scored_away,minutes_per_goal_conceded_overall,minutes_per_goal_conceded_home,minutes_per_goal_conceded_away,highest_goals_scored_home,highest_goals_scored_away,highest_goals_conceded_home,highest_goals_conceded_away,half_time_points_overall,half_time_points_home,half_time_points_away,corners_recorded_matches_num_overall,corners_recorded_matches_num_home,cornersRecorded_matches_away,corners_total_per_match_overall,corners_total_per_match_home,corners_total_per_match_away,corners_against_overall,corners_against_home,corners_against_away,corners_against_per_match_overall,corners_against_per_match_home,corners_against_per_match_away,cornerTimingRecorded_matches_overall,cornerTimingRecorded_matches_home,cornerTimingRecorded_matches_away,corners_total_fh_overall,corners_total_fh_home,corners_total_fh_away,corners_total_2h_overall,corners_total_2h_home,corners_total_2h_away,corners_total_per_match_fh_overall,corners_total_per_match_fh_home,corners_total_per_match_fh_away,corners_total_per_match_2h_overall,corners_total_per_match_2h_home,corners_total_per_match_2h_away,corners_total_fh_over4_overall,corners_total_fh_over5_overall,corners_total_fh_over6_overall,corners_total_2h_over4_overall,corners_total_2h_over5_overall,corners_total_2h_over6_overall,firstGoalScoredPercentage_overall,firstGoalScoredPercentage_home,firstGoalScoredPercentage_away,shots_per_match_overall,shots_per_match_home,shots_per_match_away,shots_on_target_per_match_overall,shots_on_target_per_match_home,shots_on_target_per_match_away,shots_off_target_per_match_overall,shots_off_target_per_match_home,shots_off_target_per_match_away,fouls_by_this_team_overall,fouls_by_this_team_home,fouls_by_this_team_away,fouls_per_match_overall,fouls_per_match_home,fouls_per_match_away,offsides_total_overall,offsides_total_home,offsides_total_away,offsides_this_team_overall,offsides_this_team_home,offsides_this_team_away,offsidesRecorded_matches_overall,offsidesRecorded_matches_home,offsidesRecorded_matches_away,offsides_total_per_match_overall,offsides_total_per_match_home,offsides_total_per_match_away,offsides_this_team_per_match_overall,offsides_this_team_per_match_home,offsides_this_team_per_match_away,offsides_total_over05_overall,offsides_total_over15_overall,offsides_total_over25_overall,offsides_total_over35_overall,offsides_total_over45_overall,scoredBothHalves_overall,scoredBothHalves_home,scoredBothHalves_away,scoredBothHalvesPercentage_overall,scoredBothHalvesPercentage_home,scoredBothHalvesPercentage_away,BTTS_both_halves_overall,BTTS_both_halves_home,BTTS_both_halves_away,BTTS_and_win_overall,BTTS_and_win_home,BTTS_and_win_away,BTTS_and_win_percentage_overall,BTTS_and_win_percentage_home,BTTS_and_win_percentage_away,BTTS_and_draw_overall,BTTS_and_draw_home,BTTS_and_draw_away,BTTS_and_draw_percentage_overall,BTTS_and_draw_percentage_home,BTTS_and_draw_percentage_away,BTTS_and_lose_overall,BTTS_and_lose_home,BTTS_and_lose_away,BTTS_and_lose_percentage_overall,BTTS_and_lose_percentage_home,BTTS_and_lose_percentage_away,matches_goal_timings_recorded_overall,matches_goal_timings_recorded_home,matches_goal_timings_recorded_away,total_goals_2h_overall,total_goals_2h_home,total_goals_2h_away,average_total_goals_2h_per_match_overall,average_total_goals_2h_per_match_home,average_total_goals_2h_per_match_away,goals_scored_2h_overall,goals_scored_2h_home,goals_scored_2h_away,goals_scored_2h_per_match_overall,goals_scored_2h_per_match_home,goals_scored_2h_per_match_away,goals_conceded_2h_overall,goals_conceded_2h_home,goals_conceded_2h_away,goals_conceded_2h_per_match_overall,goals_conceded_2h_per_match_home,goals_conceded_2h_per_match_away,points_2h_overall,points_2h_home,points_2h_away,ppg_2h_overall,ppg_2h_home,ppg_2h_away,wins_2h_overall,wins_2h_home,wins_2h_away,wins_2h_percentage_overal

In [65]:
df.head()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game
0,74439,Manchester United,Leicester City,Andre Marriner,1,0.0,0.0,1.89,1.32,2,1,3,1,1,0,2,5,2,0,1,0,0,2,0,1,8,8,7,5,1,3,11,8,46,54,0.0,0.0,1.25,1.46,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.37,4.98,9.81,1.33,2.00,3.60,7.50,2.05,1.69,Old Trafford (Manchester),premier league,2018-08-10,19:00:00,7,3,0,3,16,12,4,19,1,18/19,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0,0.0,False,False,0.0,0.0,True,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,51749,Newcastle United,Tottenham Hotspur,Martin Atkinson,1,0.0,0.0,1.32,1.74,1,2,3,3,1,2,3,5,2,0,2,0,0,2,1,1,11,13,3,6,8,7,11,12,40,60,0.0,0.0,1.18,1.58,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,4.51,3.77,1.84,1.31,1.95,3.50,5.75,1.83,1.87,St. James' Park (Newcastle upon Tyne),premier league,2018-08-11,11:30:00,8,4,0,4,24,9,15,23,2,18/19,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0,0.0,False,False,0.0,0.0,False,True,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10353,AFC Bournemouth,Cardiff City,Kevin Friend,1,0.0,0.0,1.53,0.74,2,0,2,1,1,0,7,4,1,0,1,0,1,0,1,0,11,7,5,2,6,5,11,9,62,38,0.0,0.0,1.50,0.89,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.03,3.51,3.96,1.31,1.95,3.45,6.70,1.83,1.83,Vitality Stadium (Bournemouth- Dorset),premier league,2018-08-11,14:00:00,11,2,0,2,18,7,11,20,1,18/19,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,True,False,0.0,0.0,True,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,

# Brudnopis

In [249]:
draft_data = pd.read_csv(r'data\switzerland\switzerland-super-league-matches-2017-to-2018-stats.csv')

In [250]:
draft_data.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league
0,1500742800,Jul 22 2017 - 5:00pm,complete,3415.0,Lausanne Sport,St. Gallen,NaN,1,0.0,0.0,0.89,1.28,3,3,6,3,1,2,"7,57,62","28,33,52",2,8,1,0,3,0,1,0,2,1,13,16,5,8,8,8,12,15,42,58,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.45,3.58,2.96,0.0,0.0,0.0,0.0,0.0,0.0,Stade Olympique de la Pontaise,switzerland
1,1500742800,Jul 22 2017 - 5:00pm,complete,31120.0,Young Boys,Basel,NaN,1,0.0,0.0,2.61,1.67,2,0,2,0,0,0,"58,80",NaN,1,7,3,0,6,0,1,2,3,3,8,12,3,4,5,8,15,10,40,60,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.91,3.66,2.45,0.0,0.0,0.0,0.0,0.0,0.0,STADE DE SUISSE Wankdorf Bern,switzerland
2,1500818400,Jul 23 2017 - 2:00pm,complete,20000.0,Grasshopper,Zürich,NaN,1,0.0,0.0,1.17,1.17,0,2,2,1,0,1,NaN,"23,82",3,13,2,0,3,0,1,1,1,2,8,10,4,6,4,4,14,11,46,54,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.59,3.67,2.73,0.0,0.0,0.0,0.0,0.0,0.0,Stadion Letzigrund,switzerland
3,1500818400,Jul 23 2017 - 2:00pm,complete,8553.0,Luzern,Lugano,NaN,1,0.0,0.0,1.72,1.17,1,0,1,0,0,0,81,NaN,7,2,0,0,4,0,0,0,1,3,17,13,4,6,13,7,11,14,58,42,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.91,3.87,4.11,0.0,0.0,0.0,0.0,0.0,0.0,swissporarena,switzerland
4,1500818400,Jul 23 2017 - 2:00pm,complete,5617.0,Thun,Sion,NaN,1,0.0,0.0,1.06,1.22,0,1,1,1,0,1,NaN,28,11,2,2,1,2,0,2,1,1,1,23,9,8,3,15,6,15,13,54,46,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.34,3.62,3.11,0.0,0.0,0.0,0.0,0.0,0.0,Stockhorn Arena,switzerland


In [246]:
draft_data['team_a_xg'].value_counts()

team_a_xg
1.61    8
1.51    6
1.75    6
1.99    5
2.10    5
       ..
2.07    1
2.19    1
1.44    1
1.14    1
2.01    1
Name: count, Length: 140, dtype: int64

In [260]:
[col for col in df.columns]

['attendance',
 'home_team_name',
 'away_team_name',
 'referee',
 'Game Week',
 'Pre-Match PPG (Home)',
 'Pre-Match PPG (Away)',
 'home_ppg',
 'away_ppg',
 'home_team_goal_count',
 'away_team_goal_count',
 'total_goal_count',
 'total_goals_at_half_time',
 'home_team_goal_count_half_time',
 'away_team_goal_count_half_time',
 'home_team_corner_count',
 'away_team_corner_count',
 'home_team_yellow_cards',
 'home_team_red_cards',
 'away_team_yellow_cards',
 'away_team_red_cards',
 'home_team_first_half_cards',
 'home_team_second_half_cards',
 'away_team_first_half_cards',
 'away_team_second_half_cards',
 'home_team_shots',
 'away_team_shots',
 'home_team_shots_on_target',
 'away_team_shots_on_target',
 'home_team_shots_off_target',
 'away_team_shots_off_target',
 'home_team_fouls',
 'away_team_fouls',
 'home_team_possession',
 'away_team_possession',
 'Home Team Pre-Match xG',
 'Away Team Pre-Match xG',
 'team_a_xg',
 'team_b_xg',
 'average_goals_per_match_pre_match',
 'btts_percentage_pre

In [262]:
df.tail()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_total_yellow_cards_in_home_team_games,average_total_yellow_cards_in_away_team_games,average_yellow_cards_by_home_team_in home_team_games,average_yellow_cards_by_away_team_in_away_team_games,both_teams_scored,home_team_cumulative_both_scored_pre_game,away_team_cumulative_both_scored_pre_game,home_clean_sheet,away_clean_sheet,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_win,away_win,draw,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_cumulative_shots_pre_game,home_team_cumulative_shots_on_target_pre_game,away_team_cumulative_shots_pre_game,away_team_cumulative_shots_on_target_pre_game,home_team_average_shots_per_game,home_team_average_shots_on_target_per_game,away_team_average_shots_per_game,away_team_average_shots_on_target_per_game,home_team_average_fouls_per_game,away_team_average_fouls_per_game,total_fouls,home_team_average_total_fouls_per_game,away_team_average_total_fouls_per_game,home_team_average_possession_per_game,away_team_average_possession_per_game,team_a_average_xg_per_game,team_b_average_xg_per_game,home_team_failed_to_score,away_team_failed_to_score,home_team_games_without_goals_pre_game,away_team_games_without_goals_pre_game,home_team_received_red_card,away_team_received_red_card,home_team_games_with_red_cards_pre_game,away_team_games_with_red_cards_pre_game,home_team_average_red_cards_per_game,away_team_average_red_cards_per_game,home_points,away_points,home_team_average_points_per_game,away_team_average_points_per_game
375,32140,Leicester City,Chelsea,Anthony Taylor,38,1.44,1.61,1.42,1.58,0,0,0,0,0,0,4,5,0,0,1,0,0,0,1,0,8,12,4,5,4,7,9,8,46,54,1.68,1.52,1.16,1.50,2.64,47,84,59,20,6,42,72,42,75,10.78,3.62,2.45,3.6,2.8,1.21,1.67,2.65,5.50,1.56,2.40,King Power Stadium (Leicester- Leicestershire),premier league,2019-05-12,14:00:00,9,1,0,1,20,9,11,17,0,18/19,48,39,10.891892,9.378378,5.567568,5.675676,3.486486,3.000000,1.621622,1.297297,False,22.0,16.0,True,True,9.0,15.0,False,False,True,15.0,6.0,16.0,21.0,8.0,8.0,412.0,218.0,434.0,231.0,11.135135,5.891892,11.729730,6.243243,9.243243,8.945946,17,19.864865,20.351351,51.162162,63.756757,1.514054,1.667568,True,True,7.0,9.0,False,False,5.0,